In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import json
import numpy as np
import os
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time

In [11]:
def get_keys(path):
    """
    This function will get dictionary of keys from a stored json file
    :param path: (str) directory path for the .json file with keys
    """
    with open(path) as f:
        return json.load(f)

def search_symbol(driver, ticker):
    """
    This function searches for a ticker symbol on TD Ameritrade website once
    user is logged in.
    :param driver: (Selenium webdriver) webdriver returned from start_bot()
    :param ticker: (str) ticker symbol to search
    """

    # Attempt the more expedient symbol lookup, rever to main search otherwise
    try:
        search = driver.find_element(By.XPATH, '//*[@id="symbol-lookup"]')
        search.click()
        search.clear()
    except:
        driver.switch_to.default_content()
        search = driver.find_element(By.NAME, "search")
        kind = 'search'
    else:
        kind = 'symbol'
    # Enter ticker symbol to search and click search button
    search.send_keys(ticker)
    if kind == 'symbol':
        driver.find_element(By.XPATH, '//*[@id="layout-full"]/div[1]/div/div[1]/div/a').click()
    elif kind == 'search':
        driver.find_element(By.ID ,"searchIcon").click()
    # Give extra time for webpage to load
    time.sleep(4)

def reduce_tabs(driver):
    """
    This function is used when an action opens the result on a new tab, in
    order to reduce the number of browser tabs back to 1, and switch to the
    intended tab.
    :param driver: (Selenium webdriver)
    """
    if len(driver.window_handles) > 1:
        driver.switch_to.window(driver.window_handles[0])
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

def clean(x, show_errors=False):
    """
    This function is used to clean strings containing numeric data of the 
    common issues found in TD Ameritrade's website
    """
    #print("cleaning {} of type {}".format(x, type(x)))
    not_date = True
    multiple = 1
    if isinstance(x, str):
        check = re.split('/|-|, ', x)
        x = x.strip()
        x = x.replace(',','')

        if x.startswith('(') and x.endswith(')'):
            x = x.strip('(').strip(')')
            x = '-'+x

        if x.endswith('%'):
            x = x.replace('%','')
            multiple = 1/100
        elif x.endswith('x'):
            x = x.strip('x')
            if x == '--':
                x = np.NaN
            else:
                x = x

        if x.endswith('k') or x.endswith('K'):
            x = x.upper().replace('K','')
            multiple = 1000

        if x.startswith('$') or x.startswith('-$'):
            x = x.replace('$','')
        elif len(check) > 1 and check[-1].isdigit():
            not_date = False
            if x.startswith('(Unconfirmed)'):
                x = x.replace('(Unconfirmed) ','')
            x = pd.to_datetime(x, infer_datetime_format=True)

        if x == '--':
            x = np.NaN

        if not_date:
            try:
                x = float(x) * multiple
            except:
                if show_errors:
                    print(x) 
                x = np.NaN
    #print('returning {}'.format(x))
    return x   

def start_bot(keys):
    """
    Starts TD Ameritrade Scraping Bot. Takes input of dictionary containing 
    username and password which must have keys "user" and "pass" with the 
    values to be used. Returns webdriver object to be used for session.
    :param keys: (dict) dictionary with username ("user") and password ("pass")
    """
    driver = webdriver.Chrome()
    #driver.implicitly_wait(20)
    login_url = 'https://invest.ameritrade.com/grid/p/login'
    try:
        driver.get(login_url)
    except:
        raise ValueError('Caanot find Login button')
    else:
        assert "TD Ameritrade Login" in driver.title
        WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, 'button.cafeLoginButton')).click()
        username = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.ID, 'username0'))
        username.send_keys(keys["user"])
        password = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.ID, "password1"))
        password.send_keys(keys["pass"])
        try:
            driver.find_element(By.CSS_SELECTOR, 'input#accept.accept.button').click()
        except:
            raise ValueError("Login fails.")
        else:
            try:
                WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Use desktop website'))
                time.sleep(3)
                button = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.XPATH, value='//*[@id="app"]/div/div[2]/footer/div/ul/li[1]/button'))
                button.click()
                time.sleep(3)
                home_url = driver.current_url
                reduce_tabs(driver)
            except:
                return driver

    return driver

def scrape_summary(driver, ticker, search_first=True, return_full=False, internet_speed='fast'):
    """
    This function scrapes the "Summary" tab of a TD Ameritrade security
    lookup page
    :param driver: (Selenium webdriver) webdriver returned from start_bot()
    :param ticker: (str) ticker symbol to scrape
    :param search_first: (bool) allows for chain of scrapes to be done on one
                                security when set to False. Leave set to True
                                unless you are sure you are already on the
                                desired security, or the wrong data will scrape
    :param return_full: (bool) will return dataframe with extra column containing
                               feature descriptions for the rows.
    :param internet_speed: (str) set to 'slow' if bot is not working properly due
                                to slow page loading times.
    """
    # Search symbol first if flag is True:
    if search_first:
        search_symbol(driver, ticker)
        if internet_speed == 'slow':
            time.sleep(1)
    #tabs = get_tab_links()
    #driver.get(tabs['Summary'])

    if internet_speed == 'fast':
        sleep_time = 1
    elif internet_speed == 'slow':
        sleep_time = 2

    # Find main iframe
    driver.switch_to.default_content()
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
    driver.switch_to.frame(iframes[3])

    # Switch to Summary tab
    WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Summary'))
    WebDriverWait(driver, 10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/nav/ul/li[1]/a')).click()
    
    # Wait for conditions to be met before making soup
    element = driver.find_element(By.XPATH, '//*[@id="stock-summarymodule"]/div/div/div[2]/div')
    WebDriverWait(driver, 10).until(lambda x: EC.visibility_of_element_located(element))
    # Add extra time for data to load
    time.sleep(sleep_time)
    
    # Make soup and find elements
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    dts = soup.find_all('dt')

    # Set flag which will be made false if no dividend is given
    dividend_given = True
    texts = []
    for dt in dts:
        try:
            texts.append(dt.get_text('|'))        
        except:
            print("error")
            continue

    dds = soup.find_all('dd')
    values = []
    for dd in dds:
        try:
            values.append(dd.get_text('|'))        
        except:
            print("error")
            continue

    fields = [x.split('|')[0] for x in texts]
    alt_info = [x.split('|')[1:] for x in texts]

    # Make dataframe and fix row names
    data_dict = dict(zip(fields,zip(alt_info,values)))
    temp = pd.DataFrame.from_dict(data_dict, orient='index')
    temp.loc['Volume', 1] = temp.loc['Volume', 0][0].strip()
    temp.rename(index={'Volume:':'Volume 10-day Avg',
                          'Volume':'Volume Past Day',
                          '10-day average volume:':'Volume',
                          'Score:':'New Constructs Score'
                        }, inplace=True)
    temp.loc['52-Wk Range', 1] = temp.loc['52-Wk Range', 0]
    price_feat = 'Closing Price'
    if price_feat not in temp.index:
        if 'Price' in temp.index:
            price_feat = 'Price'

    # Cleaning data
    if temp.loc["B/A Size",1] == '--': 
        temp = temp.append(pd.Series([[],
                                  np.NaN
                                 ],
                                 name="Bid Size"),
                      )
        temp = temp.append(pd.Series([[],
                                  np.NaN
                                 ],
                                 name="Ask Size"),
                      )
        temp = temp.append(pd.Series([[],
                                  np.NaN
                                 ],
                                 name="B/A Ratio"),
                      )
    else:
        temp = temp.append(pd.Series([[],
                                  float(temp.loc['B/A Size',1].split('x')[0])
                                 ],
                                 name="Bid Size"),
                      )
        temp = temp.append(pd.Series([[],
                                  float(temp.loc['B/A Size',1].split('x')[1])
                                 ],
                                 name="Ask Size"),
                      )
        temp = temp.append(pd.Series([[],
                                  float(temp.loc['B/A Size',1].split('x')[0])
                                            /float(temp.loc['B/A Size',1].split('x')[1])
                                 ],
                                 name="B/A Ratio"),
                    )  
    if temp.loc["Day's Range",1] == '--':
        temp = temp.append(pd.Series([[],
                                  np.NaN,
                                 ],
                                 name="Day Change $"
                                ),
                      )
        temp = temp.append(pd.Series([[],
                                  np.NaN
                                 ],
                                 name="Day Change %"
                                ),
                      )
        temp = temp.append(pd.Series([[],
                                 np.NaN
                                 ],
                                name="Day Low"),
                      )
        temp = temp.append(pd.Series([[],
                                 np.NaN
                                 ],
                                name="Day High"),
                      )
    else:
        temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|'))],
                                 name="Day Change $"
                                ),
                      )
        temp = temp.append(pd.Series([[],
                                  float(temp.loc["Day's Change",1].split('|')[2].strip('%)').strip('()'))/100
                                 ],
                                 name="Day Change %"
                                ),
                      )
        temp = temp.append(pd.Series([[],
                                 float(temp.loc["Day's Range",1].split('-')[0].strip('|').replace(',',''))
                                 ],
                                name="Day Low"),
                      )
        temp = temp.append(pd.Series([[],
                                 float(temp.loc["Day's Range",1].split('-')[1].strip('|').replace(',',''))
                                 ],
                                name="Day High"),
                      )
    if temp.loc["Annual Dividend/Yield",1] != 'No dividend':
        temp = temp.append(pd.Series([[],
                                 float(temp.loc["Annual Dividend/Yield",1].split('/')[0].strip('$'))
                                 ],
                                name="Annual Dividend $"))

        temp = temp.append(pd.Series([[],
                                 float(temp.loc["Annual Dividend/Yield",1].split('/')[1].strip('%'))/100
                                 ],
                                name="Annual Dividend %"))
    else:
        dividend_given = False
        temp = temp.append(pd.Series([[],
                                 np.NaN
                                 ],
                                name="Annual Dividend $"))
        temp = temp.append(pd.Series([[],
                                 np.NaN
                                 ],
                                name="Annual Dividend %"))
    temp.rename(columns={1:ticker}, inplace = True)
    drop = ["Day's Change", 
            "Day's Range",
            "Day's High",
            "Day's Low",
            "Avg Vol (10-day)", 
            #"52-Wk Range", 
            "Annual Dividend/Yield",
            "New Constructs Score"
            ]

    # Drop feature description column if flag is False (default)
    if return_full == False:
        temp.drop(index=drop, columns=[0], inplace=True, errors='ignore')
    
    # Clean data
    temp = temp.T
    # Only one of these columns will be present:
    try:
        temp['% Below High'] = temp['% Below High'].map(lambda x: float(x.strip('%'))/100, na_action='ignore')
    except:
        temp['% Above Low'] = temp['% Above Low'].map(lambda x: clean(x), na_action='ignore')
    
    temp['% Held by Institutions'] = temp['% Held by Institutions'].map(lambda x: clean(x)/100, na_action='ignore')
    temp['Short Interest'] = temp['Short Interest'].map(lambda x: clean(x)/100, na_action='ignore')
    # Set list of columns for cleaing
    try_to_clean = ['Prev Close',
                    'Ask close',
                    'Bid close',
                    'Beta',
                    'Ask',
                    'Bid',
                    'EPS (TTM, GAAP)',
                    'Last Trade',
                    'Last (size)',
                    price_feat,
                    'Historical Volatility',
                    'P/E Ratio (TTM, GAAP)',
                    "Today's Open",
                    'Volume',
                    'Volume 10-day Avg']
    # Clean columns
    for col in try_to_clean:
        try:
            temp[col] = temp[col].map(lambda x: clean(x), na_action='ignore')
        except:
            pass
    
    # Convert date info to datetime if it exists
    if dividend_given:
        try:
            temp['Ex-dividend'] = pd.to_datetime(temp['Ex-dividend Date'], infer_datetime_format=True)
        except:
            temp['Ex-dividend'] = pd.to_datetime(temp['Ex-dividend'], infer_datetime_format=True)
        temp['Dividend Pay Date'] = pd.to_datetime(temp['Dividend Pay Date'], infer_datetime_format=True)

    # Try to force any remaining numbers to floats:
    temp = temp.astype('float64', errors='ignore')
    temp = temp.T   
    temp.sort_index(inplace=True)

    return temp

def scrape_earnings(driver, ticker, search_first=True, internet_speed='fast'):
    """
    This function scrapes the "Earnings" tab of a TD Ameritrade security
    lookup page
    :param driver: (Selenium webdriver) webdriver returned from start_bot()
    :param ticker: (str) ticker symbol to scrape
    :param search_first: (bool) allows for chain of scrapes to be done on one
                                security when set to False. Leave set to True
                                unless you are sure you are already on the
                                desired security, or the wrong data will scrape
    :param internet_speed: (str) set to 'slow' if bot is not working properly due
                                to slow page loading times.
    """
    # Search for symbol if flag is True
    if search_first:
        search_symbol(driver, ticker)
        if internet_speed == 'slow':
            time.sleep(1)

    if internet_speed == 'fast':
        sleep_time = 1
    elif internet_speed == 'slow':
        sleep_time = 2
    
    # Find main iframe:  
    driver.switch_to.default_content()    
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
    driver.switch_to.frame(iframes[3])
    
    # Switch to Earnings tab:
    WebDriverWait(driver,10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/nav/ul/li[4]/a')).click()
    time.sleep(2)
    
    # Switch to Earnings Analysis (1st sub tab)
    WebDriverWait(driver,10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/div[4]/nav/nav/a[1]')).click()
    time.sleep(sleep_time)
    
    # Wait for conditions before making soup
    WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Annual Earnings History and Estimates'))
    element = driver.find_element(By.XPATH, '//*[@id="main-chart-wrapper"]')
    WebDriverWait(driver, 10).until(lambda x: EC.visibility_of_element_located(element))
    
    # Make soup and find container/elements
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    earn_dict = {}
    earnings_dict = {}
    contain = soup.find('div', {'data-module-name':'EarningsAnalysisModule'})
    header = contain.find('div', {'class':'row contain earnings-data'})
    #key = header.find('td', {'class':'label bordered'}).get_text()
    earn_dict['Next Earnings Announcement'] = header.find('td', {'class':'value week-of'}).get_text()
    
    # Get number of analysts reporting on security
    analysts = header.find_all('td', {'class':'label'})[1].get_text().split()
    for word in analysts:
        # The number of analysts will be the only numerical string
        try:
            earn_dict['Growth Analysts'] = float(word)
        except:
            continue
    # Find chart object in container, then bars
    chart = contain.find('div', {'id':'main-chart-wrapper'})
    bars = chart.find_all('div', {'class':'ui-tooltip'})
    for bar in bars:
        text = bar.get_text('|').split('|')
        # text[0] is the year
        year = text[0]
        earnings_dict[year] = {}
        # There is more text when there is a earnings surprise
        if len(text) > 4:
            earnings_dict[year]['Earnings Result'] = text[1]
            earnings_dict[year][text[2].strip('"').strip().strip(':')] = float(text[3].replace('$',''))
            earnings_dict[year][text[4].split(':')[0]] = text[4].split(':')[1].strip()
        else:
            earnings_dict[year]['Earnings Result'] = 'Neutral'
            # Should be a string: 'Actual' or 'Estimate'
            est_string = text[1].strip('"').strip().strip(':')
            # The actual consensus estimate
            est = float(text[2].replace('$',''))
            earnings_dict[year][est_string] = est
            # Should be a string: 'Estimate range'
            est_range_string = text[3].split(':')[0]
            # The estimate range as a string
            est_range = text[3].split(':')[1].strip()
            # Convert to 
            earnings_dict[year][est_range_string] = est_range
            
    # Create df and all useful columns
    earnings_yrly = pd.DataFrame.from_dict(earnings_dict, orient='index')
    earnings_yrly['Growth'] = earnings_yrly['Actual'].pct_change()
    earnings_yrly['Low Estimate'] = earnings_yrly['Estimate range'].map(lambda x: float(x.split()[0].replace('$','')), na_action='ignore')
    earnings_yrly['Low Growth Est'] = earnings_yrly['Low Estimate'].pct_change()
    earnings_yrly['High Estimate'] = earnings_yrly['Estimate range'].map(lambda x: float(x.split()[2].replace('$','')), na_action='ignore')
    earnings_yrly['High Growth Est'] = earnings_yrly['High Estimate'].pct_change()
    # Take average of high and low for years where 'Estimate' not available
    earnings_yrly['Consensus Estimate'] = (earnings_yrly['High Estimate'] + earnings_yrly['Low Estimate']) / 2
    # Supercede these values where consensus estimates are available
    idx_to_change = earnings_yrly[earnings_yrly['Estimate'].notnull()].index
    earnings_yrly.loc[idx_to_change, 'Consensus Estimate'] = earnings_yrly.loc[idx_to_change, 'Estimate']
    # Make new column that contains the actuals and consensus estimates
    earnings_yrly['Actual/Estimate'] = earnings_yrly['Actual']
    earnings_yrly.loc[idx_to_change, 'Actual/Estimate'] = earnings_yrly.loc[idx_to_change, 'Estimate']
    earnings_yrly['A/E Growth'] = earnings_yrly['Actual/Estimate'].pct_change()

    if 'Consensus estimate' in earnings_yrly.columns:
        # Sometimes ranges aren't given, and Consensus estimate given instead, fill holes caused
        earnings_yrly['Consensus Estimate'].fillna(earnings_yrly[earnings_yrly['Consensus estimate'].notnull()]['Consensus estimate'].map(lambda x: float(x.replace('$',''))), inplace=True)
        earnings_yrly.drop(columns=['Consensus estimate'], inplace=True)
    earnings_yrly.drop(columns=['Estimate range'], inplace=True)
    earnings_yrly['Consensus Growth Est'] = (earnings_yrly['High Growth Est']+earnings_yrly['Low Growth Est']) / 2
    
    low_1yr_growth_est = earnings_yrly.iloc[-2,:]['Low Growth Est']
    high_1yr_growth_est = earnings_yrly.iloc[-2,:]['High Growth Est']
    cons_1yr_growth_est = earnings_yrly.iloc[-2,:]['Consensus Growth Est']
    growth_2yr_low_est = earnings_yrly.iloc[-2:]['Low Growth Est'].mean()
    growth_2yr_high_est = earnings_yrly.iloc[-2:]['High Growth Est'].mean()
    growth_2yr_cons_est = (growth_2yr_low_est + growth_2yr_high_est) / 2
    earn_dict['Growth 1yr Low Est'] = low_1yr_growth_est
    earn_dict['Growth 1yr High Est'] = high_1yr_growth_est
    earn_dict['Growth 1yr Consensus Est'] = cons_1yr_growth_est
    earn_dict['Growth 2yr Low Est'] = growth_2yr_low_est
    earn_dict['Growth 2yr High Est'] = growth_2yr_high_est
    earn_dict['Growth 2yr Consensus Est'] = growth_2yr_cons_est
    earn_dict['Growth 5yr Low Est'] = earnings_yrly['Low Growth Est'].mean()
    earn_dict['Growth 5yr High Est'] = earnings_yrly['High Growth Est'].mean()
    earn_dict['Growth 5yr Consensus Est'] = earnings_yrly['Consensus Growth Est'].mean()
    earn_dict['Growth 5yr Actual/Est'] = earnings_yrly['A/E Growth'].mean()
    earn_dict['Growth 3yr Historic'] = earnings_yrly['Growth'].mean()

    earn_df = pd.DataFrame.from_dict(earn_dict, orient='index', columns=[ticker])
    earn_df[ticker] = earn_df[ticker].map(clean)

    return earn_df, earnings_yrly

def scrape_fundamentals(driver, ticker, search_first=True, internet_speed='fast'):
    """
    This function scrapes the "Fundamentals" tab of a TD Ameritrade security
    lookup page
    :param driver: (Selenium webdriver) webdriver returned from start_bot()
    :param ticker: (str) ticker symbol to scrape
    :param search_first: (bool) allows for chain of scrapes to be done on one
                                security when set to False. Leave set to True
                                unless you are sure you are already on the
                                desired security, or the wrong data will scrape
    :param internet_speed: (str) set to 'slow' if bot is not working properly due
                                to slow page loading times.
    """
    # Search symbol first if flag is True
    if search_first:
        search_symbol(driver, ticker)
        #tabs = get_tab_links()
    
    if internet_speed == 'fast':
        sleep_time = 1
    elif internet_speed == 'slow':
        sleep_time = 2
        time.sleep(1)

    # Gets Overview
    driver.switch_to.default_content()
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME,"iframe"))
    driver.switch_to.frame(iframes[3])
    WebDriverWait(driver,10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/nav/ul/li[5]/a')).click()
    #time.sleep(1)
    WebDriverWait(driver,10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/div[4]/nav/nav/a[1]')).click()
    time.sleep(sleep_time)
    driver.switch_to.default_content()
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
    driver.switch_to.frame(iframes[3])
    #WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Price Performance'))
    #driver.find_element_by_xpath('//*[@id="layout-full"]/nav/ul/li[5]/a').click()
    #time.sleep(1)

    # Wait for conditions before making soup
    WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Price Performance'))
    element = driver.find_element(By.XPATH, '//*[@id="price-charts-wrapper"]/div')
    WebDriverWait(driver, 10).until(lambda x: EC.visibility_of_element_located(element))
      
    # Make soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Scrapes current valuation ratios
    contain = soup.find('div', {'class': 'ui-description-list'})
    labels = [x.get_text().strip() for x in contain.find_all('dt')]
    values = [float(x.get('data-rawvalue')) for x in contain.find_all('dd')]
    for i, value in enumerate(values):
        if value == '-99999.99' or value == -99999.99:
            values[i] = np.NaN
    fundies = dict(zip(labels,values))

    # Gets 5yr low and high from chart
    contain = soup.find('div', {'class':'col-xs-8 price-history-chart'})
    five_yr = contain.find_all('div', {'class':'marker hideOnHover'})
    fundies['5yr Low'] = five_yr[0].get_text().split(' ')[1]
    fundies['5yr High'] = five_yr[1].get_text().split(' ')[1]

    # Gets 5 year Price Performance data from each hover section of graphic
    periods = contain.find_all('div', {'class':'period'})
    texts = [x.get_text('|') for x in periods]
    past_dict = {}
    yr_growths = []
    for text in texts:
        parts = text.split('|')
        year = parts[2].split(' ')[3].strip()
        past_dict[year] = {}
        high = parts[1].split(' ')[2].strip()
        low = parts[0].split(' ')[2].strip()
        change = parts[2].split(' ')[0].strip()
        past_dict[year]['high'] = high
        past_dict[year]['low'] = low
        past_dict[year]['change'] = change
        yr_growths.append(float(change.strip('%')))
    fundies['5yr Avg Return'] = np.mean(yr_growths) / 100

    # Gets Historic Growth and Share Detail
    contain = soup.find('div', {'data-module-name':'HistoricGrowthAndShareDetailModule'})
    boxes = contain.find_all('div', {'class':'col-xs-4'})
    labels = []
    values = []
    historic_data = True

    for box in boxes:
        numbers = []
        words = []
        if box.find('h4').get_text() == 'Historic Growth':
            for dt in box.find_all('dt')[1:]:
                word = dt.get_text('|').split('|')[0].strip() +' Growth 5yr'
                words.append(word)
            for dd in box.find_all('dd'):
                try:
                    number = float(dd.find('label').get('data-value'))
                    #print(number)
                    if number == -99999.99:
                        #print("here")
                        number = np.NaN
                except:
                    #print("didn't find number")
                    try:
                        number = dd.find('span').get_text()
                    except:
                        number = np.NaN
                #print(number)
                numbers.append(number)
            if len(words) == 0:
                print("Historic Growth not available for {}".format(ticker))
                historic_data = False
        else:
            for dt in box.find_all('dt')[1:]:
                word = dt.get_text('|').split('|')[0].strip()
                words.append(word)
            for dd in box.find_all('dd'):
                try:
                    number = float(dd.get('data-rawvalue'))
                    if number == -99999.99:
                        number = np.NaN
                except:
                    try:
                        number = dd.get_text()
                    except:
                        number = np.NaN
                numbers.append(number)

        labels = labels + words
        values = values + numbers
    
    # Make df of Historic Growth and Share Detail
    fundies2 = dict(zip(labels, values))

    # Get ready to scrape financial reports:
    report_names = ['Balance Sheet',
              'Income Statement',
              'Cash Flow'
             ]
    xpaths = [#'//*[@id="layout-full"]/div[4]/nav/nav/a[1]', # Already done
              '//*[@id="layout-full"]/div[4]/nav/nav/a[2]',
              '//*[@id="layout-full"]/div[4]/nav/nav/a[3]',
              '//*[@id="layout-full"]/div[4]/nav/nav/a[4]'
             ]
    reports = dict(zip(report_names, xpaths))
    
    # Function to scrape each report, since their formats are similar enough
    def scrape_report(name, xpath):
        # Switch to Appropriate Report
        driver.find_element(By.XPATH, xpath).click()
        time.sleep(sleep_time)
        iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
        driver.switch_to.frame(iframes[3])
        driver.switch_to.default_content()
        iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
        driver.switch_to.frame(iframes[3])
        WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Values displayed are in millions.'))
        element = driver.find_element(By.XPATH, '//*[@id="layout-full"]/div[4]/div/div')
        WebDriverWait(driver, 10).until(lambda x: EC.visibility_of_element_located(element))

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #pprint.pprint(soup)
        contain = soup.find('div', {'data-module-name':'FinancialStatementModule'})
        year_info = [x.get_text('|') for x in contain.find_all('th', {'scope':'col'})]
        years = [x.split('|')[0] for x in year_info]
        dates = [x.split('|')[1] for x in year_info]

        sheet = {}
        for i, year in enumerate(years):
            sheet[year] = {}
            sheet[year]['Date'] = dates[i]
        row_names = []
        contain = soup.find('div', {'class':'row contain data-view'})
        rows = contain.find_all('tr')[1:] # Skips the header row
        #rows = contain.find_all('th', {'scope':'row'})
        for row in rows:
            #print(row)
            row_name = row.get_text('|').split('|')[0]
            row_names.append(row_name)
            values = row.find_all('td')
            for i, value in enumerate(values):
                sheet[years[i]][row_name] = value.get_text()
                
        temp = pd.DataFrame.from_dict(sheet, orient='index').T
        temp['Report'] = name
        return temp
    
    
    # Create summary dataframes
    temp = pd.DataFrame.from_dict(fundies, orient='index', columns=[ticker])
    temp2 = pd.DataFrame.from_dict(fundies2, orient='index', columns=[ticker])
    temp2.rename(index={'Current Month':'Short Int Current Month',
                         'Previous Month':'Short Int Prev Month',
                         'Percent of Float':'Short Int Pct of Float'
                        },
                inplace=True)
    # Clean these rows if they exist
    try:
        temp2.loc['Short Int Pct of Float',:] = temp2.loc['Short Int Pct of Float',:].astype('float64') / 100
        temp2.loc['% Held by Institutions',:] = temp2.loc['% Held by Institutions',:].astype('float64') / 100
    except:
        print("Short Interest info not available for {}".format(ticker))

    # Create yearly dataframe
    yearly = pd.DataFrame.from_dict(past_dict, orient='index').T
    for name, xpath in reports.items():
        tempy = scrape_report(name, xpath)
        yearly = pd.concat([yearly, tempy], axis=0, sort=False)
    
    # Combine two summary dataframes
    temp = pd.concat([temp,temp2], axis=0) 

    # Clean data in the dataframes  
    for col in temp:
        temp[col] = temp[col].map(lambda x: clean(x),  na_action='ignore')
    colnames = [col for col in yearly.columns if col not in ['Report']]
    for col in colnames:
        yearly[col] = yearly[col].map(lambda x: clean(x), na_action='ignore')
    
    # Create FCF and growth features for summary from yearly:
    yearly = yearly.T.astype('float64', errors='ignore')
    temp = temp.T.astype('float64', errors='ignore')
    indices = [indx for indx in yearly.index if indx not in ['Report']]
    yearly['Free Cash Flow'] = np.NaN
    yearly['FCF Growth'] = np.NaN
    # Allows this to not throw errors if values not available
    try:
        yearly.loc[indices,'Free Cash Flow'] = yearly.loc[indices,'Total Cash from Operations'] + yearly['Capital Expenditures']
        yearly.loc[indices,'FCF Growth'] = yearly.loc[indices,'Free Cash Flow'].pct_change()
        temp['FCF Growth 5yr'] = yearly['FCF Growth'].mean()
    except:
        temp['FCF Growth 5yr'] = np.NaN
    # These percentages must be formatted
    if historic_data:
        temp['EPS Growth 5yr'] = temp['EPS Growth 5yr']/100
        temp['Revenue Growth 5yr'] = temp['Revenue Growth 5yr']/100
        temp['Dividend Growth 5yr'] = temp['Dividend Growth 5yr']/100
    else:
        temp['EPS Growth 5yr'] = np.NaN
        temp['Revenue Growth 5yr'] = np.NaN
        temp['Dividend Growth 5yr'] = np.NaN

    # Transposing dataframes back
    temp = temp.T
    yearly = yearly.T

    return temp, yearly

def scrape_valuation(driver, ticker, search_first=True, internet_speed='fast'):
    """
    This function scrapes the "Valuation" tab of a TD Ameritrade security
    lookup page
    :param driver: (Selenium webdriver) webdriver returned from start_bot()
    :param ticker: (str) ticker symbol to scrape
    :param search_first: (bool) allows for chain of scrapes to be done on one
                                security when set to False. Leave set to True
                                unless you are sure you are already on the
                                desired security, or the wrong data will scrape
    :param internet_speed: (str) set to 'slow' if bot is not working properly due
                                to slow page loading times.
    """
    # Search symbol first if flag is True
    if search_first:
        search_symbol(driver, ticker)

    if internet_speed == 'fast':
        sleep_time = 2
    elif internet_speed == 'slow':
        sleep_time = 3
        time.sleep(1)
    # Find main iframe
    driver.switch_to.default_content()
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
    driver.switch_to.frame(iframes[3])

    # Switch to Valuation tab
    WebDriverWait(driver, 10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/nav/ul/li[6]/a')).click()
    #time.sleep(1)

    # Switch to First tab under Valuation (also Valuation)
    WebDriverWait(driver,10).until(lambda x: x.find_element(By.XPATH, '//*[@id="stock-valuationmodule"]/div/div[1]/nav/a[1]')).click()
    driver.switch_to.default_content()
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
    driver.switch_to.frame(iframes[3])

    # Wait for condition before advancing
    WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, '{} vs Industry'.format(ticker)))
    
    # Prepare to scrape valuation tabs by xpath
    tab_names = ['Valuation',
                 'Profitability',
                 'Dividend',
                 'Gowth rates',
                 'Effectiveness',
                 'Financial strength'
                ]
    xpaths = ['//*[@id="stock-valuationmodule"]/div/div[1]/nav/a[1]',
              '//*[@id="stock-valuationmodule"]/div/div[1]/nav/a[2]',
              '//*[@id="stock-valuationmodule"]/div/div[1]/nav/a[3]',
              '//*[@id="stock-valuationmodule"]/div/div[1]/nav/a[4]',
              '//*[@id="stock-valuationmodule"]/div/div[1]/nav/a[5]',
              '//*[@id="stock-valuationmodule"]/div/div[1]/nav/a[6]'
             ]
    tabs = dict(zip(tab_names, xpaths))

    # Scrape each tab
    valuation_df = pd.DataFrame()
    for name, xpath in tabs.items():
        # Switch to Appropriate Report
        driver.find_element(By.XPATH, xpath).click()
        iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
        driver.switch_to.frame(iframes[3])
        driver.switch_to.default_content()
        iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
        driver.switch_to.frame(iframes[3])
        WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, '{} vs Industry'.format(ticker)))
        element = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.XPATH, '//*[@id="stock-valuationmodule"]/div/div[1]/div[2]'))
        WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, '{} Analysis'.format(name)))
        time.sleep(sleep_time)
        # Prevents breaking when there is no info on a tab, by waiting for condition
        try:
            element = driver.find_element(By.XPATH, '//*[@id="stock-valuationmodule"]/div/div/div[2]/table/tbody/tr[1]/td[2]')
            WebDriverWait(driver, 10).until(lambda x: EC.visibility_of_element_located(element))
        except:
            continue

        # Make soup and find container
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        contain = soup.find('div', {'data-module-name':'StocksValuationModule'})
        
        # Get data
        row_names = [x.get_text() for x in contain.find_all('a', {'class':'definition-link'})]
        tds = soup.find_all('td', {'class':'data-compare'})
        value_dict = {}
        for i, row_name in enumerate(row_names[1:]):
            value_dict[row_name] = {}
            dts = tds[i].find_all('dt')
            dds = tds[i].find_all('dd')
            cols = [dt.get_text() for dt in dts]
            vals = [dd.get_text() for dd in dds]
            value_dict[row_name][cols[0]] = vals[0]
            value_dict[row_name][cols[1]] = vals[1]
            value_dict[row_name]['Type'] = name

        # Create dataframe
        temp = pd.DataFrame.from_dict(value_dict, orient='columns').T
        valuation_df = pd.concat([valuation_df, temp], axis=0, sort=False)
    
    # Clean all columns except 'Type'
    for col in valuation_df.columns:
        if col != 'Type':
            valuation_df[col] = valuation_df[col].apply(lambda x: clean(x))

    # Create ratio to industry feature for normalized feature
    valuation_df['Ratio to Industry'] = valuation_df[ticker] / valuation_df['Industry']
    
    return valuation_df

def scrape_analysts(driver, ticker, search_first=True, internet_speed='fast'):
    """
    This function scrapes the "Analyst Reports" tab of a TD Ameritrade security
    lookup page
    :param driver: (Selenium webdriver) webdriver returned from start_bot()
    :param ticker: (str) ticker symbol to scrape
    :param search_first: (bool) allows for chain of scrapes to be done on one
                                security when set to False. Leave set to True
                                unless you are sure you are already on the
                                desired security, or the wrong data will scrape
    :param internet_speed: (str) set to 'slow' if bot is not working properly due
                                to slow page loading times.
    """
    # Search symbol first if flag is True
    if search_first:
        search_symbol(driver, ticker)

        if internet_speed == 'slow':
            time.sleep(1)

    if internet_speed == 'fast':
        sleep_time = 1
    elif internet_speed == 'slow':
        sleep_time = 2
    # Find iframe with tabs (main iframe)
    driver.switch_to.default_content()
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
    driver.switch_to.frame(iframes[3])

    # Switch to Analyst Reports tab
    WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Summary'))
    driver.find_element(By.XPATH, '//*[@id="layout-full"]/nav/ul/li[8]/a').click()
    time.sleep(sleep_time)

    # Wait for conditions before soup is made
    driver.switch_to.default_content()
    iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
    driver.switch_to.frame(iframes[3])
    WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Archived Reports'))

    # Make soup and find container and elements
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    contain = soup.find('table', {'class':'ui-table provider-table'}).find('tbody')
    trs = contain.find_all('tr')

    analysts = []
    ratings = []
    dates = []
    for tr in trs:
        divs = tr.find_all('div')
        analyst = divs[0].get('class')[1].strip()
        
        try:
            # Skip vickers
            if analyst == 'vickers':
                continue
            # Special treatment for marketEdge
            else:
                # Get date or NaN otherwise
                try:
                    txt = tr.find('p', {'class':'rating-since'}).get_text()
                    date = txt.replace('Rating Since ','')
                except:
                    date = np.NaN
                # Special treatment for marketEdge
                if analyst == 'marketEdge':
                    analysts.append(analyst+' opinion')
                    rating = divs[2].get('class')[2]
                    ratings.append(rating)
                    dates.append(date)
                    flag = False
                    i = 0
                    while flag == False:
                        i += 1
                        rating = divs[3].get('class')[1][-i].strip()
                        try:
                            rating = float(rating)
                            if i != 1:
                                rating = -rating
                            flag = True
                        except:
                            flag = False
                # Special treatment for cfra
                elif analyst == 'cfra':
                    rating = divs[2].get('class')[1][-1].strip()
                    try:
                        int(rating)
                    except:
                        rating = np.NaN
                else:
                    rating = divs[2].get('class')[1].strip()
                # Try to make ratings numeric
                try:
                    rating = int(rating)
                except:
                    rating = rating
        except:
            rating = np.NaN
            date = np.NaN

        analysts.append(analyst)
        ratings.append(rating)
        dates.append(date)

    # Create dataframe
    analyst_dict = dict(zip(analysts,zip(ratings,dates)))
    temp = pd.DataFrame.from_dict(analyst_dict, 
                                  orient='index', 
                                  columns=[ticker,'Rating Since'],
                                  )
    # Convert date column to datetime
    temp['Rating Since'] = pd.to_datetime(temp['Rating Since'], infer_datetime_format=True)
    
    return temp

def scrape_ticker(driver, ticker, errors='ignore', internet_speed='fast'):
    """
    This function scrapes every tab of a security based on ticker passed.
    Each scrape will be attempted 5 times before being skipped, as it is 
    unlikely for the data to fail to scrape this many times unless it is 
    truly absent.
    :param driver: (Selenium webdriver) webdriver returned from start_bot()
    :param ticker: (str) ticker symbol to scrape
    :param internet_speed: (str) set to 'slow' if bot is not working properly due
                                to slow page loading times.
    """
    # Getting Summary
    success = False
    tries = 0
    while not success:
        tries += 1
        try:
            summary = scrape_summary(driver, ticker, internet_speed=internet_speed)
            success = True
        except:
            print("Failed to gather summary for {} on attempt {}".format(ticker, tries))
        if tries >= 5:    
            print("Too many failed attempts for summary of {}, skipping to next df.".format(ticker))
            summary = pd.DataFrame(columns=[ticker])
            if errors == 'raise':
                raise
            elif errors == 'ignore':
                break

    # Getting Earnings
    success = False
    tries = 0
    while not success:
        tries += 1
        try:
            earnings, earnings_yearly = scrape_earnings(driver, ticker, search_first=False, internet_speed=internet_speed)
            success = True
        except:
            print("Failed to gather earnings for {} on attempt {}".format(ticker, tries))
        if tries >= 5:
            print("Too many failed attempts for earnings of {}, skipping to next df.".format(ticker))
            earnings = pd.DataFrame(columns=[ticker])
            earnings_yearly = pd.DataFrame(columns=[ticker])
            if errors == 'raise':
                raise
            elif errors == 'ignore':
                break
    
    # Getting fundamentals
    success = False
    tries = 0
    while not success:
        tries += 1
        try:
            fundies, fundies_yearly = scrape_fundamentals(driver, ticker, search_first=False, internet_speed=internet_speed)
            success = True
        except:
            print("Failed to gather fundamentals for {} on attempt {}".format(ticker, tries))
        if tries >= 5:
            print("Too many failed attempts for fundamentals of {}, skipping to next df.".format(ticker))
            fundies = pd.DataFrame(columns=[ticker])
            fundies_yearly = pd.DataFrame(columns=[ticker])
            if errors == 'raise':
                raise
            elif errors == 'ignore':
                break

    # Getting valuation
    success = False
    tries = 0
    while not success:
        tries += 1
        try:
            valuation = scrape_valuation(driver, ticker, search_first=False, internet_speed=internet_speed)
            success = True
        except:
            print("Failed to gather valuation for {} on attempt {}".format(ticker, tries))
        if tries >= 5:
            print("Too many failed attempts for valuation of {}, skipping to next df.".format(ticker))
            valuation = pd.DataFrame(columns=[ticker])
            if errors == 'raise':
                raise
            elif errors == 'ignore':
                break
    
    # Getting analyst reports
    success = False
    tries = 0
    while not success:
        tries += 1
        try:
            analysis = scrape_analysts(driver, ticker, search_first=False, internet_speed=internet_speed)
            success = True
        except:
            print("Failed to gather analysts for {} on attempt {}".format(ticker, tries))
        if tries >= 5:
            print("Too many failed attempts for analysts of {}, skipping to next df.".format(ticker))
            analysis = pd.DataFrame(columns=[ticker])
            if errors == 'raise':
                raise
            elif errors == 'ignore':
                break
    
    # Create combined 1D df for later stacking
    combined = pd.concat([summary[ticker].drop(index=['Shares Outstanding']),
                          earnings[ticker],
                          fundies[ticker],
                          valuation[ticker],
                          analysis[ticker]
                         ],
                        axis=0)
    # Remove duplicate rows from combined
    combined = pd.DataFrame(combined.loc[~combined.index.duplicated(keep='first')])
    for analyst in analysis.index:
        combined.loc[analyst+' since'] = analysis.loc[analyst, 'Rating Since']
    # Produce dictionary of results
    results = {'combined':combined, 
               'summary':summary, 
               'earnings':earnings, 
               'earnings_yearly':earnings_yearly, 
               'fundies':fundies, 
               'fundies_yearly':fundies_yearly, 
               'valuation':valuation,
               'analysts':analysis
              }
    return results

def scrape_watchlist(driver, tickers, name, root_dir='', skip_finished=True,
                     save_df=False, errors='ignore', return_skipped=False,
                     internet_speed='fast'):
    """
    Main wrapper function for scraper. Can do large lists of securities,
    and will store the data into assigned directory (can be set with kwarg)
    :param driver: selenium webdriver
    :param tickers: (list) ticker symbols
    :param name: (str) name of watchlist
    :param root_dir: (str) directory to save database to. Will use current working
                            directory if none passed.
    :param save_df: (bool) Whether to save the combined df to disk
    :param errors: (str) 'raise' or 'ignore'
    :param return_skipped: (bool) can return list of skipped securities if
                            ignoring errors
    :param internet_speed: (str) set to 'slow' if bot is not working properly due
                            to slow page loading times.
    """
    # Make list for skipped securities if needed
    if return_skipped == True:
        skipped = []

    # Create path name based on date and watchlist name, and make directory
    # path_name = root_dir + name + '_' + datetime.today().strftime('%m-%d-%Y')
    path_name = root_dir + name + '_' + datetime(2022, 11, 27).strftime('%m-%d-%Y')
    if not os.path.isdir(path_name):
        os.mkdir(path_name)
    
    # Create empty dataframe
    big_df = pd.DataFrame()
    
    # Scrape each ticker
    for i, ticker in enumerate(tickers):
        tickers_done = i + 1
        # Establish ticker path
        ticker_path = path_name+'/{}'.format(ticker)
        
        # Skip previously scraped securities if flag is True
        if skip_finished:
            if os.path.isdir(ticker_path):
                continue
        
        # Scrape security
        try:
            results = scrape_ticker(driver, ticker, errors=errors, internet_speed=internet_speed)
        except:
            print("Did not successfully scrape {}".format(ticker))
            if errors == 'raise':
                raise
            else:
                if return_skipped:
                    skipped.append(ticker)
                continue
        
        # Make directory if there is none
        if not os.path.isdir(ticker_path):
            os.mkdir(ticker_path)
        
        # Dump .csv files to directory
        for name, dataframe in results.items():
            try:
                dataframe.to_csv(ticker_path + '/{}'.format(name) + '.csv')
            except:
                print("No {} dataframe for {}".format(name,ticker))
        
        # Compile security to big_df
        big_df = pd.concat([big_df, results['combined'].T], axis=0, sort=True)
        
        # Print number of tickers completed every 10 completions
        if tickers_done % 10 == 0:
            print("{} tickers scraped".format(tickers_done))

    # Saves combined dataframe to file if called
    if save_df:
        big_df.to_csv(path_name + '/{}'.format('big_df.csv'))
    
    if not return_skipped:
        return big_df,
    else:
        return big_df, skipped

def build_big_df(tickers, database_path):
    """
    This function reads a previously scraped watchlist database at the provided
    path, and combines all of the 'combined.csv' files into one dataframe.
    :param tickers: (list-like) The securities to be gathered
    :param database_path: (str) The location of the database
    """
    big_df = pd.DataFrame()
    for ticker in tickers:
        file_path = database_path+'/{}/combined.csv'.format(ticker)
        try:
            temp = pd.read_csv(file_path, index_col='Unnamed: 0').T
        except:
            temp = pd.DataFrame(pd.read_csv(file_path)).T
        big_df = pd.concat([big_df, temp.astype('float64',errors='ignore')], axis=0, sort=True)
    new_df = pd.DataFrame()
    for col in big_df:
        new_df[col] = big_df[col].astype('float64', copy=True, errors='ignore')
    for col in new_df.columns:
        if col.endswith('since'):
            new_df[col] = pd.to_datetime(new_df[col], infer_datetime_format=True)
    
    return new_df

In [5]:
ticker_lists = pd.read_csv('nmr_us_tickers.csv')

In [4]:
driver = start_bot(get_keys('tda_keys.json'))

In [98]:
results = scrape_ticker(driver, 'NVDA', internet_speed='slow')

C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1707361144.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1707361144.py:247: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1707361144.py:252: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1707361144.py:282: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip(

In [108]:
results.keys()

dict_keys(['combined', 'summary', 'earnings', 'earnings_yearly', 'fundies', 'fundies_yearly', 'valuation', 'analysts'])

In [5]:
df, skipped = scrape_watchlist(driver, ticker_lists.yahoo.tolist(), 'nmr_us', root_dir='', skip_finished=True, return_skipped=True, internet_speed='slow')

Failed to gather summary for BRKS on attempt 1
Failed to gather summary for BRKS on attempt 2
Failed to gather summary for BRKS on attempt 3
Failed to gather summary for BRKS on attempt 4
Failed to gather summary for BRKS on attempt 5
Too many failed attempts for summary of BRKS, skipping to next df.
Failed to gather earnings for BRKS on attempt 1
Failed to gather earnings for BRKS on attempt 2
Failed to gather earnings for BRKS on attempt 3
Failed to gather earnings for BRKS on attempt 4
Failed to gather earnings for BRKS on attempt 5
Too many failed attempts for earnings of BRKS, skipping to next df.
Failed to gather fundamentals for BRKS on attempt 1
Failed to gather fundamentals for BRKS on attempt 2
Failed to gather fundamentals for BRKS on attempt 3
Failed to gather fundamentals for BRKS on attempt 4
Failed to gather fundamentals for BRKS on attempt 5
Too many failed attempts for fundamentals of BRKS, skipping to next df.
Failed to gather valuation for BRKS on attempt 1
Failed to

C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:247: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:252: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:282: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|')

Failed to gather summary for OMI on attempt 2


C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:247: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:252: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:282: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|')

Failed to gather summary for OMI on attempt 3


...

C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:247: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:252: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:282: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|')

Failed to gather fundamentals for ZTS on attempt 1


C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:247: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:252: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:282: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|')

Historic Growth not available for ZWS


C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:247: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:252: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_7664\2121936164.py:282: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|')

In [9]:
skipped.info()

AttributeError: 'list' object has no attribute 'info'

In [2]:
skipped

NameError: name 'skipped' is not defined

In [8]:
df

,% Above Low,% Below High,% Held by Institutions,52-Wk Range,5yr Avg Return,5yr High,5yr Low,Annual Dividend $,Annual Dividend %,Annual Dividend Yield,...,marketEdge,marketEdge opinion,marketEdge opinion since,marketEdge since,newConstructs,newConstructs since,researchTeam,researchTeam since,theStreet,theStreet since
WTFC,0.1549,NaN,0.9153,[76.13 - 105.56],0.020683,105.56,22.02,1.36,0.0151,0.0151,...,NaN,NaN,NaT,NaT,rating-box,2022-11-28,rating-box,2022-10-21,rating-box,2021-05-11
WTRG,NaN,0.1161,0.767,[38.50 - 53.93],0.055567,54.52,30.4,1.15,0.0236,0.0236,...,NaN,NaN,NaT,NaT,rating-box,2022-11-28,rating-box,2022-02-18,rating-box,2020-12-27
WTS,0.3075,NaN,0.9541,[116.31 - 201.00],0.17145,212.0,61.17,1.2,0.0077,0.0077,...,NaN,NaN,NaT,NaT,rating-box,2022-11-26,rating-box,2021-04-26,rating-box,2018-05-04
WTTR,NaN,0.2272,0.6313,[5.54 - 10.43],0.034317,21.96,2.22,0.2,0.0249,0.0249,...,NaN,NaN,NaT,NaT,rating-box,2022-11-26,rating-box,2022-11-04,rating-box,2022-11-04
WTW,NaN,0.0211,0.9551,[187.89 - 244.73],0.072867,271.87,134.5,3.28,0.0134,0.0134,...,NaN,NaN,NaT,NaT,rating-box,2022-11-26,rating-box,2022-08-20,rating-box,2021-12-27
WU,0.1627,NaN,1.0081,[12.27 - 20.40],-0.021783,28.45,12.27,0.94,0.0658,0.0658,...,NaN,NaN,NaT,NaT,rating-box,2022-11-28,rating-box,2022-08-20,rating-box,2022-07-13
WW,0.1287,NaN,0.8176,[3.42 - 19.07],-0.2673,105.73,3.42,NaN,NaN,NaN,...,NaN,NaN,NaT,NaT,rating-box,2022-11-28,rating-box,2022-10-30,rating-box,2022-10-30
WWD,0.1927,NaN,0.8323,[79.26 - 129.12],0.06155,130.75,46.51,0.76,0.0077,0.0077,...,NaN,NaN,NaT,NaT,rating-box,2022-11-28,rating-box,2022-08-20,rating-box,2022-02-01
WWE,NaN,0.0102,1.168,[46.91 - 81.63],0.29585,100.45,27.81,0.48,0.0059,0.0059,...,NaN,NaN,NaT,NaT,rating-box,2022-11-28,rating-box,2022-11-25,rating-box,2022-05-17
WWW,0.0549,NaN,0.9872,[10.39 - 33.41],-0.10525,44.74,10.39,0.4,0.0356,0.0356,...,NaN,NaN,NaT,NaT,rating-box,2022-11-28,rating-box,2021-08-06,rating-box,2021-06-25


In [7]:
ticker_lists

,yahoo
0,A
1,AA
2,AADI
3,AAL
4,AAN
...,...
2210,ZTS
2211,ZUMZ
2212,ZUO
2213,ZWS


In [8]:
ticker_lists['yahoo']

0          A
1         AA
2       AADI
3        AAL
4        AAN
        ... 
2210     ZTS
2211    ZUMZ
2212     ZUO
2213     ZWS
2214    ZYXI
Name: yahoo, Length: 2215, dtype: object

In [3]:
skipped = ['BRKS', 'BF-B', 'BRK-B', 'LGF-A', 'MOG-A', 'OMI']

In [6]:
ticker_lists.loc[ticker_lists.yahoo.isin(skipped)]

,yahoo
232,BRKS
263,BF-B
321,BRK-B
1167,LGF-A
1292,MOG-A
1459,OMI


In [4]:
dir_path = 'nmr_us_11-27-2022'

In [7]:
ticker_lists_no_skipped = ticker_lists.loc[~ticker_lists.yahoo.isin(skipped)]

In [8]:
ticker_lists_no_skipped.reset_index(drop=True).to_csv('nmr_us_tickers_11-27-2022.csv', index=False)

In [12]:
nmr_us = build_big_df(ticker_lists_no_skipped.yahoo, dir_path)

C:\Users\USER\AppData\Local\Temp\ipykernel_19736\1961211346.py:1216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[col] = big_df[col].astype('float64', copy=True, errors='ignore')
C:\Users\USER\AppData\Local\Temp\ipykernel_19736\1961211346.py:1216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[col] = big_df[col].astype('float64', copy=True, errors='ignore')
C:\Users\USER\AppData\Local\Temp\ipykernel_19736\1961211346.py:1216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling 

In [26]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

In [35]:
nmr_us.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Index: 2209 entries, A to ZYXI
Data columns (total 104 columns):
 #    Column                                             Non-Null Count  Dtype         
---   ------                                             --------------  -----         
 0    % Above Low                                        1297 non-null   float64       
 1    % Below High                                       912 non-null    float64       
 2    % Held by Institutions                             2198 non-null   float64       
 3    52-Wk Range                                        2209 non-null   object        
 4    5yr Avg Return                                     2190 non-null   float64       
 5    5yr High                                           2190 non-null   float64       
 6    5yr Low                                            2190 non-null   float64       
 7    Annual Dividend $                                  997 non-null    float64       
 8    Annual Divi

In [55]:
nmr_us.select_dtypes(include='object').columns

Index(['52-Wk Range', 'B/A Size', 'Change Since Close', 'Dividend Pay Date',
       'Ex-dividend', 'Ex-dividend Date', 'Last (time)', 'Market Cap',
       'Market Edge Opinion:', 'Next Earnings Announcement',
       'P/E Ratio (TTM, GAAP)', 'Volume Past Day', 'cfra', 'ford',
       'marketEdge', 'newConstructs', 'researchTeam', 'theStreet'],
      dtype='object')

In [49]:
nmr_us['Volume Past Day'].value_counts()

(Light)            1898
(Average)           111
(Below Average)      90
(Heavy Day)          84
(Above Average)      26
Name: Volume Past Day, dtype: int64

In [64]:
nmr_us['52-Wk Range'].head()

A       ['112.52 - 162.62']
AA        ['33.55 - 98.09']
AADI      ['11.00 - 26.60']
AAL       ['11.65 - 21.42']
AAN        ['7.65 - 25.96']
Name: 52-Wk Range, dtype: object

In [92]:
nmr_us['52-Wk Range'].map(lambda x: x.strip("[]'").replace(' ', '').split('-'))

A       [112.52, 162.62]
AA        [33.55, 98.09]
AADI      [11.00, 26.60]
AAL       [11.65, 21.42]
AAN        [7.65, 25.96]
              ...       
ZTS     [124.15, 249.27]
ZUMZ      [20.29, 49.98]
ZUO        [6.14, 20.43]
ZWS       [21.39, 37.77]
ZYXI       [4.97, 14.55]
Name: 52-Wk Range, Length: 2209, dtype: object

In [109]:
nmr_us[nmr_us['Market Cap'].map(lambda x: 'T' in x)]

,% Above Low,% Below High,% Held by Institutions,52-Wk Range,5yr Avg Return,5yr High,5yr Low,Annual Dividend $,Annual Dividend %,Annual Dividend Yield,Ask,Ask Size,Ask close,B/A Ratio,B/A Size,Beta,Bid,Bid Size,Bid close,Change Since Close,Change in Debt/Total Capital Quarter over Quarter,Closing Price,Day Change $,Day Change %,Day High,Day Low,Days to Cover,Dividend Change %,Dividend Growth 5yr,"Dividend Growth Rate, 3 Years",Dividend Pay Date,"EPS (TTM, GAAP)",EPS Growth (MRQ),EPS Growth (TTM),EPS Growth 5yr,Ex-dividend,Ex-dividend Date,FCF Growth 5yr,Float,Gross Profit Margin (TTM),Growth 1yr Consensus Est,Growth 1yr High Est,Growth 1yr Low Est,Growth 2yr Consensus Est,Growth 2yr High Est,Growth 2yr Low Est,Growth 3yr Historic,Growth 5yr Actual/Est,Growth 5yr Consensus Est,Growth 5yr High Est,Growth 5yr Low Est,Growth Analysts,Historical Volatility,Institutions Holding Shares,Interest Coverage (MRQ),Last (size),Last (time),Last Trade,Market Cap,Market Edge Opinion:,Net Profit Margin (TTM),Next Earnings Announcement,Operating Profit Margin (TTM),"P/E Ratio (TTM, GAAP)","PEG Ratio (TTM, GAAP)",Prev Close,Price,Price/Book (MRQ),Price/Cash Flow (TTM),Price/Earnings (TTM),"Price/Earnings (TTM, GAAP)",Price/Sales (TTM),Quick Ratio (MRQ),Return On Assets (TTM),Return On Equity (TTM),Return On Investment (TTM),Revenue Growth (MRQ),Revenue Growth (TTM),Revenue Growth 5yr,Revenue Per Employee (TTM),Shares Outstanding,Short Int Current Month,Short Int Pct of Float,Short Int Prev Month,Short Interest,Today's Open,Total Debt/Total Capital (MRQ),Volume,Volume 10-day Avg,Volume Past Day,cfra,cfra since,ford,ford since,marketEdge,marketEdge opinion,marketEdge opinion since,marketEdge since,newConstructs,newConstructs since,researchTeam,researchTeam since,theStreet,theStreet since
AAPL,0.1478,NaN,0.5798,['129.04 - 182.94'],0.290700,182.94,35.50,0.92,0.0062,0.0062,148.12,800.0,NaN,5.875,4700x800,1.2,148.11,4700.0,NaN,NaN,-0.0005,148.11,-2.96,-0.0196,148.88,147.12,1.21464,0.0588,0.084472,0.0627,2022-11-10 00:00:00,6.10,0.0753,0.0883,0.215750,2022-11-04 00:00:00,11/04/22,0.369541,15896.63,0.4331,0.028248,0.092210,-0.035714,0.060196,0.081767,0.038624,0.199873,0.228001,0.226432,0.236640,0.216224,39.0,0.355,5465.0,336.41,50.0,1:00p ET 11/25/22,NaN,2.4T,Neutral From Avoid,0.2531,2023-01-26 00:00:00,0.3029,24.27,2.73,151.07,NaN,46.60129,21.24439,24.27197,24.27,5.97511,0.85,0.2836,1.7546,0.4704,0.0866,0.0779,0.114591,2404439.0,15908.120,114431895.0,0.007199,103178670.0,0.0072,148.31,0.7032,35195860.0,72685349.0,(Light),NaN,2017-08-02,rating-box,2022-10-21,rating-box,NaN,NaT,2009-04-13,rating-box,2022-11-26,rating-box,2022-10-21,NaN,NaT
GOOGL,0.1694,NaN,0.7615,['83.34 - 151.55'],0.151883,151.55,48.88,NaN,NaN,NaN,97.37,400.0,NaN,0.500,200x400,1.1,97.33,200.0,NaN,NaN,0.0001,97.46,-1.00,-0.0102,98.64,97.40,1.24539,NaN,NaN,NaN,No dividend,4.94,-0.1213,-0.0353,0.321133,NaN,No dividend,0.208875,11592.08,0.5610,-0.957107,-0.957027,-0.957187,-0.457131,-0.363759,-0.550504,0.276433,0.264844,-0.457131,-0.363759,-0.550504,42.0,0.379,4855.0,15.59,500.0,1:00p ET 11/25/22,NaN,1.3T,Neutral From Avoid,0.2375,2023-01-31 00:00:00,0.2785,19.73,2.20,98.46,NaN,4.98432,15.36439,19.72627,19.73,4.47437,2.47,0.1899,0.2689,0.2318,-0.0085,0.1794,0.233363,1802639.0,12943.000,53117948.0,0.009003,51989928.0,0.0090,98.24,0.0561,9701441.0,28516948.0,(Light),NaN,2022-04-27,rating-box,2022-04-08,NaN,NaN,NaT,NaT,rating-box,2022-11-26,rating-box,2022-04-08,rating-box,2018-04-24
MSFT,0.1596,NaN,0.7009,['213.43 - 344.30'],0.237217,349.67,80.70,2.72,0.0110,0.0110,245.79,100.0,262.85,1.000,100x100,0.9,245.51,100.0,247.43,-1.84| |(-0.74%),-0.0012,247.49,NaN,NaN,NaN,NaN,1.01466,0.1050,0.096036,0.1037,2022-12-08 00:00:00,9.28,0.0514,0.0374,0.242745,2022-11-16 00:00:00,11/16/22,0.122475,7449.03,0.6826,0.031239,0.058140,0.004338,0.108625,0.152446,0.064804,0.135417,0.187316,0.194917,0.215847,0.173987,24.0,0.350,5914.0,NaN,8.0,1:00p ET 11/25/22,245.65,1.8T,Neutral From Avoid,0.3437,2023-01-24 00:0

In [118]:
nmr_us['marketEdge opinion']

0.0

In [58]:
nmr_us.loc['NVDA']

% Above Low                            0.5047
% Below High                              NaN
% Held by Institutions                 0.6318
52-Wk Range               ['108.13 - 334.12']
5yr Avg Return                        0.39695
                                 ...         
newConstructs since       2022-11-28 00:00:00
researchTeam                       rating-box
researchTeam since        2022-08-26 00:00:00
theStreet                          rating-box
theStreet since           2022-09-16 00:00:00
Name: NVDA, Length: 104, dtype: object

In [58]:
nmr_us[nmr_us['PEG Ratio (TTM, GAAP)'] < 1]

,% Above Low,% Below High,% Held by Institutions,52-Wk Range,5yr Avg Return,5yr High,5yr Low,Annual Dividend $,Annual Dividend %,Annual Dividend Yield,Ask,Ask Size,Ask close,B/A Ratio,B/A Size,Beta,Bid,Bid Size,Bid close,Change Since Close,Change in Debt/Total Capital Quarter over Quarter,Closing Price,Day Change $,Day Change %,Day High,Day Low,Days to Cover,Dividend Change %,Dividend Growth 5yr,"Dividend Growth Rate, 3 Years",Dividend Pay Date,"EPS (TTM, GAAP)",EPS Growth (MRQ),EPS Growth (TTM),EPS Growth 5yr,Ex-dividend,Ex-dividend Date,FCF Growth 5yr,Float,Gross Profit Margin (TTM),Growth 1yr Consensus Est,Growth 1yr High Est,Growth 1yr Low Est,Growth 2yr Consensus Est,Growth 2yr High Est,Growth 2yr Low Est,Growth 3yr Historic,Growth 5yr Actual/Est,Growth 5yr Consensus Est,Growth 5yr High Est,Growth 5yr Low Est,Growth Analysts,Historical Volatility,Institutions Holding Shares,Interest Coverage (MRQ),Last (size),Last (time),Last Trade,Market Cap,Market Edge Opinion:,Net Profit Margin (TTM),Next Earnings Announcement,Operating Profit Margin (TTM),"P/E Ratio (TTM, GAAP)","PEG Ratio (TTM, GAAP)",Prev Close,Price,Price/Book (MRQ),Price/Cash Flow (TTM),Price/Earnings (TTM),"Price/Earnings (TTM, GAAP)",Price/Sales (TTM),Quick Ratio (MRQ),Return On Assets (TTM),Return On Equity (TTM),Return On Investment (TTM),Revenue Growth (MRQ),Revenue Growth (TTM),Revenue Growth 5yr,Revenue Per Employee (TTM),Shares Outstanding,Short Int Current Month,Short Int Pct of Float,Short Int Prev Month,Short Interest,Today's Open,Total Debt/Total Capital (MRQ),Volume,Volume 10-day Avg,Volume Past Day,cfra,cfra since,ford,ford since,marketEdge,marketEdge opinion,marketEdge opinion since,marketEdge since,newConstructs,newConstructs since,researchTeam,researchTeam since,theStreet,theStreet since
ABG,NaN,0.1097,1.0572,['138.88 - 203.92'],0.210267,230.97,39.36,NaN,NaN,NaN,182.23,NaN,NaN,NaN,--,1.1,NaN,NaN,NaN,NaN,0.0002,181.56,-0.34,-0.0019,183.30,179.98,11.42292,NaN,NaN,NaN,No dividend,35.10,0.0179,0.4143,0.290574,NaN,No dividend,-0.026798,21.97417,0.2020,1.990233,1.984615,1.995851,0.904297,0.967179,0.841416,0.369785,0.421239,0.536921,0.574618,0.499225,9.0,0.506,362.0,7.74,10.0,5:00p ET 11/25/22,NaN,4.0B,Long,0.0546,2023-02-14 00:00:00,0.0842,5.17,0.28,181.90,NaN,1.52048,4.73040,5.17239,5.17,0.27940,0.85,0.1378,0.3979,0.1640,-0.0213,0.5273,0.085489,1012866.0,22.13306,1980255.0,0.090116,1989796.0,0.0901,179.98,0.5593,39325.0,145095.0,(Light),NaN,2020-07-14,rating-box,2022-10-21,rating-box,NaN,NaT,2020-07-15,rating-box,2022-11-25,rating-box,2021-07-23,NaN,NaT
ACLS,NaN,0.0520,0.8845,['46.41 - 83.74'],0.262233,83.74,12.99,NaN,NaN,NaN,79.92,800.0,NaN,0.125000,100x800,1.6,78.84,100.0,NaN,NaN,-0.0023,79.38,0.31,0.0039,79.59,78.05,2.67814,NaN,NaN,NaN,No dividend,4.80,-0.0837,1.1213,0.519423,NaN,No dividend,0.525222,32.19281,0.4441,2.686926,2.510949,2.862903,1.372571,1.312647,1.432495,0.708333,0.888021,0.915047,0.875098,0.954997,6.0,0.685,383.0,239.65,1.0,1:00p ET 11/25/22,NaN,2.6B,Long,0.1883,2023-02-06 00:00:00,0.2360,16.55,0.83,79.07,NaN,4.25204,15.04856,16.54529,16.55,3.03331,2.64,0.2001,0.2873,0.2544,0.0362,0.4848,0.199311,766160.0,32.84862,1131722.0,0.035121,1257567.0,0.0351,78.17,0.0706,93576.0,362111.0,(Light),NaN,NaT,rating-box,2022-09-09,rating-box,NaN,NaT,2020-05-14,rating-box,2022-11-25,rating-box,2021-08-06,NaN,NaT
AEL,NaN,0.1252,0.9878,['28.05 - 44.49'],0.054817,44.49,9.07,0.36,0.0092,0.0092,39.24,NaN,NaN,NaN,--,1.0,NaN,NaN,NaN,NaN,0.0070,38.92,-0.11,-0.0028,39.41,38.81,1.13859,0.0625,0.072145,0.0669,2022-12-13 00:00:00,13.70,-0.0905,2.8155,0.361788,2022-11-25 00:00:00,11/25/22,1.238036,84.18821,NaN,NaN,NaN,NaN,0.229200,0.260870,0.197531,0.554740,0.699917,0.229200,0.260870,0.197531,3.0,0.468,346.0,NaN,34.0,5:00p ET 11/25/22,NaN,3.3B,Neutral From Long,0.7125,2023-02-16 00:00:00,-0.4145,2.84,0.12,39.03,NaN,1.04307,2.55940,2.84085,2.84,1.78341,NaN,0.0179,0.2690,NaN,3.0517,-0.4819,0.106907,2337885.0,85.70226,1240879.0,0.014748,1265860.0,0.0147,39.33,0.213

In [44]:
nmr_us[(nmr_us['Growth 1yr Consensus Est'] > 0) & (nmr_us['Growth 2yr Consensus Est'] > 0)]

,% Above Low,% Below High,% Held by Institutions,52-Wk Range,5yr Avg Return,5yr High,5yr Low,Annual Dividend $,Annual Dividend %,Annual Dividend Yield,Ask,Ask Size,Ask close,B/A Ratio,B/A Size,Beta,Bid,Bid Size,Bid close,Change Since Close,Change in Debt/Total Capital Quarter over Quarter,Closing Price,Day Change $,Day Change %,Day High,Day Low,Days to Cover,Dividend Change %,Dividend Growth 5yr,"Dividend Growth Rate, 3 Years",Dividend Pay Date,"EPS (TTM, GAAP)",EPS Growth (MRQ),EPS Growth (TTM),EPS Growth 5yr,Ex-dividend,Ex-dividend Date,FCF Growth 5yr,Float,Gross Profit Margin (TTM),Growth 1yr Consensus Est,Growth 1yr High Est,Growth 1yr Low Est,Growth 2yr Consensus Est,Growth 2yr High Est,Growth 2yr Low Est,Growth 3yr Historic,Growth 5yr Actual/Est,Growth 5yr Consensus Est,Growth 5yr High Est,Growth 5yr Low Est,Growth Analysts,Historical Volatility,Institutions Holding Shares,Interest Coverage (MRQ),Last (size),Last (time),Last Trade,Market Cap,Market Edge Opinion:,Net Profit Margin (TTM),Next Earnings Announcement,Operating Profit Margin (TTM),"P/E Ratio (TTM, GAAP)","PEG Ratio (TTM, GAAP)",Prev Close,Price,Price/Book (MRQ),Price/Cash Flow (TTM),Price/Earnings (TTM),"Price/Earnings (TTM, GAAP)",Price/Sales (TTM),Quick Ratio (MRQ),Return On Assets (TTM),Return On Equity (TTM),Return On Investment (TTM),Revenue Growth (MRQ),Revenue Growth (TTM),Revenue Growth 5yr,Revenue Per Employee (TTM),Shares Outstanding,Short Int Current Month,Short Int Pct of Float,Short Int Prev Month,Short Interest,Today's Open,Total Debt/Total Capital (MRQ),Volume,Volume 10-day Avg,Volume Past Day,cfra,cfra since,ford,ford since,marketEdge,marketEdge opinion,marketEdge opinion since,marketEdge since,newConstructs,newConstructs since,researchTeam,researchTeam since,theStreet,theStreet since
A,NaN,0.0348,0.8930,['112.52 - 162.62'],0.159983,179.57,60.42,0.90,0.0057,0.0057,156.96,100.0,NaN,5.000,500x100,1.1,152.40,500.0,NaN,NaN,0.0003,156.96,1.61,0.0104,156.96,154.71,2.27041,0.0814,0.100275,0.0873,2023-01-25 00:00:00,4.18,0.1223,0.0613,0.147804,2022-12-30 00:00:00,12/30/22,0.805355,294.17500,0.5435,0.145669,0.156863,0.134474,0.120040,0.127584,0.112496,0.131484,0.176900,0.156938,0.177692,0.136184,17.0,0.350,1333.0,27.56,30.0,5:00p ET 11/25/22,NaN,46.5B,Long,0.1831,2023-02-21 00:00:00,0.2363,37.55,3.02,155.35,NaN,8.75462,29.47371,37.55455,37.55,6.76157,1.47,0.1182,0.2349,0.1421,0.0763,0.0837,0.088961,NaN,295.00000,2889384.0,0.009829,2893740.0,0.0098,155.09,0.3436,642657.0,1531121.0,(Light),NaN,2021-08-18,rating-box,2022-05-06,rating-box,NaN,NaT,2016-03-28,rating-box,2022-11-25,rating-box,2022-05-06,NaN,NaT
AADI,0.2109,NaN,0.6271,['11.00 - 26.60'],-0.102367,82.50,6.30,NaN,NaN,NaN,20.73,200.0,NaN,2.500,500x200,1.3,12.85,500.0,NaN,NaN,-0.0100,13.32,0.15,0.0114,13.43,13.25,10.56765,NaN,NaN,NaN,No dividend,-2.97,0.2189,0.7442,NaN,NaN,No dividend,0.233978,20.63551,0.8077,3.956897,3.879310,4.034483,1.933734,1.815980,2.051488,1.612193,1.389829,0.966867,0.907990,1.025744,4.0,0.638,91.0,NaN,7.0,1:00p ET 11/25/22,NaN,324.9M,NaN,-5.6957,2023-03-16 00:00:00,-5.7471,NaN,NaN,13.17,NaN,1.92151,NaN,NaN,NaN,29.56984,10.57,-0.3450,-0.3911,-0.3754,0.2351,90.5750,NaN,281769.0,24.39512,1061468.0,0.054996,937861.0,0.0550,13.30,0.0000,28331.0,83176.0,(Light),NaN,NaT,rating-box,2022-11-18,NaN,NaN,NaT,NaT,rating-box,NaT,rating-box,NaT,rating-box,NaT
AAON,NaN,0.0624,0.7017,['47.50 - 83.79'],0.153133,83.79,29.05,0.48,0.0061,0.0061,126.28,200.0,NaN,1.500,300x200,0.8,70.56,300.0,NaN,NaN,0.0750,78.56,0.15,0.0019,79.48,78.43,2.80083,0.0000,0.096262,0.0590,2022-12-16 00:00:00,1.26,0.7134,-0.0585,0.018337,2022-11-25 00:00:00,11/25/22,-1.753292,42.26687,0.2409,0.192745,0.215278,0.170213,0.303083,0.336210,0.269955,0.051262,0.276465,0.254329,0.267620,0.241038,3.0,0.384,283.0,38.47,21.0,1:00p ET 11/25/22,NaN,4.2B,Long,0.0878,2023-03-01 00:00:00,0.1111,62.56,NaN,78.41,NaN,7.98035,41.33924,62.55972,62.56,5.42441,1.12,0.1039,0.1464,0.1246,0.1618,0.4963,0.068394,267432.0,53.19954,666129.0,0.01576

In [24]:
nmr_us.loc['NVDA']

% Above Low                            0.5047
% Below High                              NaN
% Held by Institutions                 0.6318
52-Wk Range               ['108.13 - 334.12']
5yr Avg Return                        0.39695
                                 ...         
newConstructs since       2022-11-28 00:00:00
researchTeam                       rating-box
researchTeam since        2022-08-26 00:00:00
theStreet                          rating-box
theStreet since           2022-09-16 00:00:00
Name: NVDA, Length: 104, dtype: object

In [111]:
results['fundies']

,NVDA
Price/Earnings (TTM),7.026312e+01
Price/Sales (TTM),1.422556e+01
Price/Book (MRQ),1.909639e+01
Price/Cash Flow (TTM),5.503350e+01
5yr Low,3.112000e+01
5yr High,3.464700e+02
5yr Avg Return,3.931667e-01
EPS Growth 5yr,4.306999e-01
Revenue Growth 5yr,3.125022e-01
Dividend Growth 5yr,5.703080e-02


In [36]:
summary = scrape_summary(driver, 'NVDA')

C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1979482013.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1979482013.py:247: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1979482013.py:252: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_18168\1979482013.py:282: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip(

In [92]:
yearning, yearning_yrly = scrape_earnings(driver, 'NVDA', search_first=False, internet_speed='slow')

In [22]:
fundies, fundies_yrly = scrape_fundamentals(driver, 'NVDA', search_first=False)

In [26]:
valuation = scrape_valuation(driver, 'NVDA')

In [29]:
analysis = scrape_analysts(driver, 'NVDA')

In [93]:
yearning

,NVDA
Next Earnings Announcement,2023-02-22 00:00:00
Growth Analysts,35.0
Growth 1yr Low Est,-0.293981
Growth 1yr High Est,-0.244851
Growth 1yr Consensus Est,-0.269416
Growth 2yr Low Est,-0.09781
Growth 2yr High Est,0.165453
Growth 2yr Consensus Est,0.033821
Growth 5yr Low Est,0.001648
Growth 5yr High Est,0.139304


In [94]:
yearning_yrly

,Earnings Result,Actual,Estimate,Growth,Low Estimate,Low Growth Est,High Estimate,High Growth Est,Consensus Estimate,Actual/Estimate,A/E Growth,Consensus Growth Est
Year ending 1/2020,Neutral,1.45,NaN,NaN,5.50,NaN,5.72,NaN,5.610,1.45,NaN,NaN
Year ending 1/2021,Neutral,2.50,NaN,0.724138,9.65,0.754545,10.31,0.802448,9.980,2.50,0.724138,0.778497
Year ending 1/2022,Neutral,4.44,NaN,0.776000,4.32,-0.552332,4.37,-0.576140,4.345,4.44,0.776000,-0.564236
Year ending 1/2023,Neutral,NaN,3.26,0.000000,3.05,-0.293981,3.30,-0.244851,3.260,3.26,-0.265766,-0.269416
Year ending 1/2024,Neutral,NaN,4.34,0.000000,3.35,0.098361,5.20,0.575758,4.340,4.34,0.331288,0.337059


In [30]:
analysis

,NVDA,Rating Since
newConstructs,rating-box,2022-11-22
researchTeam,rating-box,2022-08-26
theStreet,rating-box,2022-09-16
cfra,NaN,2020-03-28
ford,rating-box,2022-11-11
marketEdge opinion,NaN,NaT


In [27]:
valuation

,NVDA,Industry,Type,Ratio to Industry
"Price/Earnings (TTM, GAAP)",6.515000e+01,20.0800,Valuation,3.244522
Price/Sales (TTM),1.319000e+01,7.0900,Valuation,1.860367
Price/Book (MRQ),1.771000e+01,12.3700,Valuation,1.431690
"PEG Ratio (TTM, GAAP)",3.060000e+00,0.9900,Valuation,3.090909
Gross Profit Margin (TTM),5.784000e-01,0.5469,Profitability,1.057597
Operating Profit Margin (TTM),2.079000e-01,0.2539,Profitability,0.818826
Net Profit Margin (TTM),2.085000e-01,0.2198,Profitability,0.948590
Annual Dividend Yield,1.000000e-03,0.0150,Dividend,0.066667
Dividend Change %,0.000000e+00,0.0694,Dividend,0.000000
"Dividend Growth Rate, 3 Years",1.610000e-02,0.0917,Dividend,0.175573


In [23]:
fundies

,NVDA
Price/Earnings (TTM),6.515045e+01
Price/Sales (TTM),1.319044e+01
Price/Book (MRQ),1.770685e+01
Price/Cash Flow (TTM),5.102901e+01
5yr Low,3.112000e+01
5yr High,3.464700e+02
5yr Avg Return,3.916667e-01
EPS Growth 5yr,4.306999e-01
Revenue Growth 5yr,3.125022e-01
Dividend Growth 5yr,5.703080e-02


In [24]:
fundies_yrly

,2017,2018,2019,2020,2021,2022,Q4 2022,Q1 2023,Q2 2023,Q3 2023,Report
high,54.34,73.19,60.45,147.27,346.47,307.11,NaN,NaN,NaN,NaN,NaN
low,45.15,31.12,31.92,45.17,115.67,108.13,NaN,NaN,NaN,NaN,NaN
change,-0.1025,-0.3181,0.8011,1.1872,1.2444,-0.4621,NaN,NaN,NaN,NaN,NaN
Date,NaN,NaN,NaN,NaN,NaN,NaN,2022-01-30 00:00:00,2022-05-01 00:00:00,2022-07-31 00:00:00,2022-10-30 00:00:00,Balance Sheet
Cash & Equivalents,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,3887.0,3013.0,13143.0,Balance Sheet
...,...,...,...,...,...,...,...,...,...,...,...
NetCash-BeginBal/RsvdforFutUse,NaN,NaN,NaN,NaN,NaN,NaN,847.0,1990.0,1990.0,1990.0,Cash Flow
NetCash-EndBal/RsrvforFutUse,NaN,NaN,NaN,NaN,NaN,NaN,1990.0,3887.0,3013.0,2800.0,Cash Flow
"Depreciation, Supplemental",NaN,NaN,NaN,NaN,NaN,NaN,1174.0,334.0,712.0,1118.0,Cash Flow
Free Cash Flow,NaN,NaN,NaN,NaN,NaN,NaN,8132.0,1370.0,2207.0,2069.0,NaN


In [25]:
search_symbol(driver, 'NVDA')

In [77]:
# Find main iframe:  
driver.switch_to.default_content()    
iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, "iframe"))
driver.switch_to.frame(iframes[3])

# Switch to Earnings tab:
WebDriverWait(driver,10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/nav/ul/li[4]/a')).click()
time.sleep(1.5)

# Switch to Earnings Analysis (1st sub tab)
WebDriverWait(driver,10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/div[4]/nav/nav/a[1]')).click()
time.sleep(1.5)

# Wait for conditions before making soup
WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Annual Earnings History and Estimates'))
element = driver.find_element(By.XPATH, '//*[@id="main-chart-wrapper"]')
WebDriverWait(driver, 10).until(lambda x: EC.visibility_of_element_located(element))

# Make soup and find container/elements
soup = BeautifulSoup(driver.page_source, 'html.parser')
earn_dict = {}
earnings_dict = {}
contain = soup.find('div', {'data-module-name':'EarningsAnalysisModule'})
header = contain.find('div', {'class':'row contain earnings-data'})
#key = header.find('td', {'class':'label bordered'}).get_text()
earn_dict['Next Earnings Announcement'] = header.find('td', {'class':'value week-of'}).get_text()

In [71]:
earn_dict

{'Next Earnings Announcement': 'February 22, 2023'}

In [72]:
header

<div class="row contain earnings-data"><table><thead><tr><th colspan="4">Annual Earnings Met Consensus (Year ending 01/2022)</th><th colspan="2">Next Earnings Announcement</th></tr></thead><tbody><tr><td class="label">Annual <div class="ui-tooltip ui-on-click ui-definition-link"><div class="closer" data-clickaction="openTooltip"></div><a class="definition-link" data-clickaction="openTooltip" data-sitevendortrack='{"actionName": "unaffiliatedLinkClick", "details": ["definition", "GAAP vs. Non-GAAP Earnings"]}' href="#">Non-GAAP earnings</a><div class="ui-tooltip-content shadow boxshadow ui-open-up ui-position-left click"><h4 class="heading heading-3">GAAP vs. Non-GAAP Earnings</h4><p>TD Ameritrade displays two types of stock earnings numbers, which are calculated differently and may report different values for the same period. GAAP earnings are the official numbers reported by a company, and non-GAAP earnings are adjusted to be more readable in earnings history and forecasts.</p><div cl

In [78]:
# Get number of analysts reporting on security
analysts = header.find_all('td', {'class':'label'})[1].get_text().split()
for word in analysts:
    # The number of analysts will be the only numerical string
    try:
        earn_dict['Growth Analysts'] = float(word)
    except:
        continue
# Find chart object in container, then bars
chart = contain.find('div', {'id':'main-chart-wrapper'})
bars = chart.find_all('div', {'class':'ui-tooltip'})
for bar in bars:
    text = bar.get_text('|').split('|')
    # text[0] is the year
    year = text[0]
    earnings_dict[year] = {}
    # There is more text when there is a earnings surprise
    if len(text) > 4:
        earnings_dict[year]['Earnings Result'] = text[1]
        earnings_dict[year][text[2].strip('"').strip().strip(':')] = float(text[3].replace('$',''))
        earnings_dict[year][text[4].split(':')[0]] = text[4].split(':')[1].strip()
    else:
        earnings_dict[year]['Earnings Result'] = 'Neutral'
        # Should be a string: 'Actual' or 'Estimate'
        est_string = text[1].strip('"').strip().strip(':')
        # The actual consensus estimate
        est = float(text[2].replace('$',''))
        earnings_dict[year][est_string] = est
        # Should be a string: 'Estimate range'
        est_range_string = text[3].split(':')[0]
        # The estimate range as a string
        est_range = text[3].split(':')[1].strip()
        # Convert to 
        earnings_dict[year][est_range_string] = est_range


In [79]:
chart

<div class="col-xs-12 chart-wrapper currently-loading" data-chartinputs='{"type":"earnings","wsodissue":"218647","period":"ann","fyEndMonth":1,"isReport":false}' data-loading="1" data-symbol="NVDA" id="main-chart-wrapper"><img src="/grid/wwws/research/uploadhandler/z2d997d0az62481e728a3d4ce38ca05491d2bd3fcb.gif"/><div class="ui-tooltip" style="position: absolute; top: 135px; left: 64px; height: 20px; width: 62px;"><div class="earnings-chart-tooltip ui-allow-hover" data-mouseenteraction="openTooltip" data-mouseleaveaction="openTooltip" style="position: absolute; height: 20px; width: 62px;"></div><div class="ui-tooltip-content shadow ui-open-up ui-position-left"><header><h3>Year ending 1/2020</h3></header><p>Actual: <strong>$1.45</strong></p><p>Estimate range: $5.50 to $5.72</p></div><div class="ui-arrow"></div></div><div class="ui-tooltip" style="position: absolute; top: 121px; left: 254px; height: 34px; width: 62px;"><div class="earnings-chart-tooltip ui-allow-hover" data-mouseenteract

In [80]:
bars

[<div class="ui-tooltip" style="position: absolute; top: 135px; left: 64px; height: 20px; width: 62px;"><div class="earnings-chart-tooltip ui-allow-hover" data-mouseenteraction="openTooltip" data-mouseleaveaction="openTooltip" style="position: absolute; height: 20px; width: 62px;"></div><div class="ui-tooltip-content shadow ui-open-up ui-position-left"><header><h3>Year ending 1/2020</h3></header><p>Actual: <strong>$1.45</strong></p><p>Estimate range: $5.50 to $5.72</p></div><div class="ui-arrow"></div></div>,
 <div class="ui-tooltip" style="position: absolute; top: 121px; left: 254px; height: 34px; width: 62px;"><div class="earnings-chart-tooltip ui-allow-hover" data-mouseenteraction="openTooltip" data-mouseleaveaction="openTooltip" style="position: absolute; height: 34px; width: 62px;"></div><div class="ui-tooltip-content shadow ui-open-up ui-position-center"><header><h3>Year ending 1/2021</h3></header><p>Actual: <strong>$2.50</strong></p><p>Estimate range: $9.65 to $10.31</p></div><d

In [29]:
earn_dict

{'Next Earnings Announcement': 'February 22, 2023', 'Growth Analysts': 35.0}

In [12]:
earnings_dict

{}

In [81]:
# Create df and all useful columns
earnings_yrly = pd.DataFrame.from_dict(earnings_dict, orient='index')
earnings_yrly['Growth'] = earnings_yrly['Actual'].pct_change()
earnings_yrly['Low Estimate'] = earnings_yrly['Estimate range'].map(lambda x: float(x.split()[0].replace('$','')), na_action='ignore')
earnings_yrly['Low Growth Est'] = earnings_yrly['Low Estimate'].pct_change()
earnings_yrly['High Estimate'] = earnings_yrly['Estimate range'].map(lambda x: float(x.split()[2].replace('$','')), na_action='ignore')
earnings_yrly['High Growth Est'] = earnings_yrly['High Estimate'].pct_change()
# Take average of high and low for years where 'Estimate' not available
earnings_yrly['Consensus Estimate'] = (earnings_yrly['High Estimate'] + earnings_yrly['Low Estimate']) / 2
# Supercede these values where consensus estimates are available
idx_to_change = earnings_yrly[earnings_yrly['Estimate'].notnull()].index
earnings_yrly.loc[idx_to_change, 'Consensus Estimate'] = earnings_yrly.loc[idx_to_change, 'Estimate']
# Make new column that contains the actuals and consensus estimates
earnings_yrly['Actual/Estimate'] = earnings_yrly['Actual']
earnings_yrly.loc[idx_to_change, 'Actual/Estimate'] = earnings_yrly.loc[idx_to_change, 'Estimate']
earnings_yrly['A/E Growth'] = earnings_yrly['Actual/Estimate'].pct_change()

if 'Consensus estimate' in earnings_yrly.columns:
    # Sometimes ranges aren't given, and Consensus estimate given instead, fill holes caused
    earnings_yrly['Consensus Estimate'].fillna(earnings_yrly[earnings_yrly['Consensus estimate'].notnull()]['Consensus estimate'].map(lambda x: float(x.replace('$',''))), inplace=True)
    earnings_yrly.drop(columns=['Consensus estimate'], inplace=True)
earnings_yrly.drop(columns=['Estimate range'], inplace=True)
earnings_yrly['Consensus Growth Est'] = (earnings_yrly['High Growth Est']+earnings_yrly['Low Growth Est']) / 2

low_1yr_growth_est = earnings_yrly.iloc[-2,:]['Low Growth Est']
high_1yr_growth_est = earnings_yrly.iloc[-2,:]['High Growth Est']
cons_1yr_growth_est = earnings_yrly.iloc[-2,:]['Consensus Growth Est']
growth_2yr_low_est = earnings_yrly.iloc[-2:]['Low Growth Est'].mean()
growth_2yr_high_est = earnings_yrly.iloc[-2:]['High Growth Est'].mean()
growth_2yr_cons_est = (growth_2yr_low_est + growth_2yr_high_est) / 2
earn_dict['Growth 1yr Low Est'] = low_1yr_growth_est
earn_dict['Growth 1yr High Est'] = high_1yr_growth_est
earn_dict['Growth 1yr Consensus Est'] = cons_1yr_growth_est
earn_dict['Growth 2yr Low Est'] = growth_2yr_low_est
earn_dict['Growth 2yr High Est'] = growth_2yr_high_est
earn_dict['Growth 2yr Consensus Est'] = growth_2yr_cons_est
earn_dict['Growth 5yr Low Est'] = earnings_yrly['Low Growth Est'].mean()
earn_dict['Growth 5yr High Est'] = earnings_yrly['High Growth Est'].mean()
earn_dict['Growth 5yr Consensus Est'] = earnings_yrly['Consensus Growth Est'].mean()
earn_dict['Growth 5yr Actual/Est'] = earnings_yrly['A/E Growth'].mean()
earn_dict['Growth 3yr Historic'] = earnings_yrly['Growth'].mean()

earn_df = pd.DataFrame.from_dict(earn_dict, orient='index', columns=['NVDA'])
earn_df['NVDA'] = earn_df['NVDA'].map(clean)

In [82]:
earnings_dict

{'Year ending 1/2020': {'Earnings Result': 'Neutral',
  'Actual': 1.45,
  'Estimate range': '$5.50 to $5.72'},
 'Year ending 1/2021': {'Earnings Result': 'Neutral',
  'Actual': 2.5,
  'Estimate range': '$9.65 to $10.31'},
 'Year ending 1/2022': {'Earnings Result': 'Neutral',
  'Actual': 4.44,
  'Estimate range': '$4.32 to $4.37'},
 'Year ending 1/2023': {'Earnings Result': 'Neutral',
  'Estimate': 3.26,
  'Estimate range': '$3.05 to $3.30'},
 'Year ending 1/2024': {'Earnings Result': 'Neutral',
  'Estimate': 4.34,
  'Estimate range': '$3.35 to $5.20'}}

In [83]:
earnings_yrly

,Earnings Result,Actual,Estimate,Growth,Low Estimate,Low Growth Est,High Estimate,High Growth Est,Consensus Estimate,Actual/Estimate,A/E Growth,Consensus Growth Est
Year ending 1/2020,Neutral,1.45,NaN,NaN,5.50,NaN,5.72,NaN,5.610,1.45,NaN,NaN
Year ending 1/2021,Neutral,2.50,NaN,0.724138,9.65,0.754545,10.31,0.802448,9.980,2.50,0.724138,0.778497
Year ending 1/2022,Neutral,4.44,NaN,0.776000,4.32,-0.552332,4.37,-0.576140,4.345,4.44,0.776000,-0.564236
Year ending 1/2023,Neutral,NaN,3.26,0.000000,3.05,-0.293981,3.30,-0.244851,3.260,3.26,-0.265766,-0.269416
Year ending 1/2024,Neutral,NaN,4.34,0.000000,3.35,0.098361,5.20,0.575758,4.340,4.34,0.331288,0.337059


In [353]:
login_url = 'https://invest.ameritrade.com/grid/p/login'
driver = webdriver.Chrome()

In [354]:
tda_login = get_keys('tda_keys.json')

try:
    driver.get(login_url)
except:
    raise ValueError('Caanot find Login button')
else:
    assert "TD Ameritrade Login" in driver.title
    WebDriverWait(driver, 10).until(lambda x: x.find_element(By.CSS_SELECTOR, 'button.cafeLoginButton')).click()
    username = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.ID, 'username0'))
    username.send_keys(tda_login['user'])
    password = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.ID, "password1"))
    password.send_keys(tda_login['pass'])    
    try:
        driver.find_element(By.CSS_SELECTOR, 'input#accept.accept.button').click()
    except:
        raise ValueError("Login fails.")
    else:
        try:
            WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Use desktop website'))
            time.sleep(3)
            button = WebDriverWait(driver, 10).until(lambda x: x.find_element(By.XPATH, value='//*[@id="app"]/div/div[2]/footer/div/ul/li[1]/button'))
            button.click()
            time.sleep(3)
            home_url = driver.current_url
            reduce_tabs(driver)
        except:
            driver.switch_to.default_content()

In [355]:
driver.switch_to.default_content()
search = driver.find_element(By.NAME, "search")
kind = 'search'

In [356]:
search.send_keys('NVDA')
if kind == 'symbol':
    driver.find_element(By.XPATH, '//*[@id="layout-full"]/div[1]/div/div[1]/div/a').click()
elif kind == 'search':
    driver.find_element(By.ID, 'searchIcon').click()
time.sleep(4)

In [357]:
# Find main iframe
driver.switch_to.default_content()
iframes = WebDriverWait(driver, 10).until(lambda x: x.find_elements(By.TAG_NAME, 'iframe'))
driver.switch_to.frame(iframes[3])

In [358]:
 # Switch to Summary tab
WebDriverWait(driver, 10).until(lambda x: EC.text_to_be_present_in_element(x, 'Summary'))
WebDriverWait(driver, 10).until(lambda x: x.find_element(By.XPATH, '//*[@id="layout-full"]/nav/ul/li[1]/a')).click()


In [359]:
# Wait for conditions to be met before making soup
element = driver.find_element(By.XPATH, '//*[@id="stock-summarymodule"]/div/div/div[2]/div')
WebDriverWait(driver, 10).until(lambda x: EC.visibility_of_element_located(element))
# Add extra time for data to load
time.sleep(1)

In [185]:
# Make soup and find elements
soup = BeautifulSoup(driver.page_source, 'html.parser')
dts = soup.find_all('dt')

# Set flag which will be made false if no dividend is given
dividend_given = True
texts = []
for dt in dts:
    try:
        texts.append(dt.get_text('|'))        
    except:
        print("error")
        continue

dds = soup.find_all('dd')
values = []
for dd in dds:
    try:
        values.append(dd.get_text('|'))        
    except:
        print("error")
        continue

fields = [x.split('|')[0] for x in texts]
alt_info = [x.split('|')[1:] for x in texts]

In [207]:
# Make dataframe and fix row names
data_dict = dict(zip(fields,zip(alt_info,values)))
temp = pd.DataFrame.from_dict(data_dict, orient='index')
temp.loc['Volume', 1] = temp.loc['Volume', 0][0].strip()
temp.rename(index={'Volume:':'Volume 10-day Avg',
                        'Volume':'Volume Past Day',
                        '10-day average volume:':'Volume',
                        'Score:':'New Constructs Score'
                    }, inplace=True)
temp.loc['52-Wk Range', 1] = temp.loc['52-Wk Range', 0]
price_feat = 'Closing Price'

In [208]:
temp

,0,1
Closing Price,[],$154.09
Day's Change,[],-2.68| |(-1.71%)
Bid,[],153.57
Ask,[],153.60
B/A Size,[],500x900
Day's High,[],160.02
Day's Low,[],151.20
Volume Past Day,"[(Below Average), Volume:, 49,207,165, 10-day ...",(Below Average)
Volume 10-day Avg,[],"57,771,720"
Volume,[],"49,207,165"


In [197]:
temp

,0,1
Closing Price,[],$154.09
Day's Change,[],-2.68| |(-1.71%)
Bid,[],153.57
Ask,[],153.60
B/A Size,[],500x900
Day's High,[],160.02
Day's Low,[],151.20
Volume,"[(Below Average), Volume:, 49,207,165, 10-day ...","49,207,165"
Volume:,[],"57,771,720"
10-day average volume:,[],"49,207,165"


In [342]:
if price_feat not in temp.index:
    if 'Price' in temp.index:
        price_feat = 'Price'

# Cleaning data
if temp.loc["B/A Size",1] == '--': 
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                                name="Bid Size"),
                    )
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                                name="Ask Size"),
                    )
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                                name="B/A Ratio"),
                    )
else:
    temp = temp.append(pd.Series([[],
                                float(temp.loc['B/A Size',1].split('x')[0])
                                ],
                                name="Bid Size"),
                    )
    temp = temp.append(pd.Series([[],
                                float(temp.loc['B/A Size',1].split('x')[1])
                                ],
                                name="Ask Size"),
                    )
    temp = temp.append(pd.Series([[],
                                float(temp.loc['B/A Size',1].split('x')[0])
                                        /float(temp.loc['B/A Size',1].split('x')[1])
                                ],
                                name="B/A Ratio"),
                )  
if temp.loc["Day's Range",1] == '--':
    temp = temp.append(pd.Series([[],
                                np.NaN,
                                ],
                                name="Day Change $"
                            ),
                    )
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                                name="Day Change %"
                            ),
                    )
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                            name="Day Low"),
                    )
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                            name="Day High"),
                    )
else:
    temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|'))],
                                name="Day Change $"
                            ),
                    )
    temp = temp.append(pd.Series([[],
                                float(temp.loc["Day's Change",1].split('|')[2].strip('%)').strip('()'))/100
                                ],
                                name="Day Change %"
                            ),
                    )
    temp = temp.append(pd.Series([[],
                                float(temp.loc["Day's Range",1].split('-')[0].strip('|').replace(',',''))
                                ],
                            name="Day Low"),
                    )
    temp = temp.append(pd.Series([[],
                                float(temp.loc["Day's Range",1].split('-')[1].strip('|').replace(',',''))
                                ],
                            name="Day High"),
                    )
if temp.loc["Annual Dividend/Yield",1] != 'No dividend':
    temp = temp.append(pd.Series([[],
                                float(temp.loc["Annual Dividend/Yield",1].split('/')[0].strip('$'))
                                ],
                            name="Annual Dividend $"))

    temp = temp.append(pd.Series([[],
                                float(temp.loc["Annual Dividend/Yield",1].split('/')[1].strip('%'))/100
                                ],
                            name="Annual Dividend %"))
else:
    dividend_given = False
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                            name="Annual Dividend $"))
    temp = temp.append(pd.Series([[],
                                np.NaN
                                ],
                            name="Annual Dividend %"))
temp.rename(columns={1:'NVDA'}, inplace = True)
drop = ["Day's Change", 
        "Day's Range",
        "Day's High",
        "Day's Low",
        "Avg Vol (10-day)", 
        #"52-Wk Range", 
        "Annual Dividend/Yield",
        "New Constructs Score"
        ]

# Drop feature description column if flag is False (default)
if False == False:
    temp.drop(index=drop, columns=[0], inplace=True, errors='ignore')

# Clean data
temp = temp.T
# Only one of these columns will be present:
try:
    temp['% Below High'] = temp['% Below High'].map(lambda x: float(x.strip('%'))/100, na_action='ignore')
except:
    temp['% Above Low'] = temp['% Above Low'].map(lambda x: clean(x), na_action='ignore')

temp['% Held by Institutions'] = temp['% Held by Institutions'].map(lambda x: clean(x)/100, na_action='ignore')
temp['Short Interest'] = temp['Short Interest'].map(lambda x: clean(x)/100, na_action='ignore')
# Set list of columns for cleaing
try_to_clean = ['Prev Close',
                'Ask close',
                'Bid close',
                'Beta',
                'Ask',
                'Bid',
                'EPS (TTM, GAAP)',
                'Last Trade',
                'Last (size)',
                price_feat,
                'Historical Volatility',
                'P/E Ratio (TTM, GAAP)',
                "Today's Open",
                'Volume',
                'Volume 10-day Avg']
# Clean columns
for col in try_to_clean:
    try:
        temp[col] = temp[col].map(lambda x: clean(x), na_action='ignore')
    except:
        pass

# Convert date info to datetime if it exists
if dividend_given:
    try:
        temp['Ex-dividend'] = pd.to_datetime(temp['Ex-dividend Date'], infer_datetime_format=True)
    except:
        temp['Ex-dividend'] = pd.to_datetime(temp['Ex-dividend'], infer_datetime_format=True)
    temp['Dividend Pay Date'] = pd.to_datetime(temp['Dividend Pay Date'], infer_datetime_format=True)

# Try to force any remaining numbers to floats:
temp = temp.astype('float64', errors='ignore')
temp = temp.T   
temp.sort_index(inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_13904\442169927.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_13904\442169927.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_13904\442169927.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],
C:\Users\USER\AppData\Local\Temp\ipykernel_13904\442169927.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = temp.append(pd.Series([[],float(temp.loc["Day's Change",1].split('|')[0].strip('|'))],


In [343]:
temp

,NVDA
% Above Low,0.425
% Held by Institutions,0.6248
52-Wk Range,[108.13 - 346.47]
Annual Dividend $,0.16
Annual Dividend %,0.001
Ask,153.6
Ask Size,900.0
B/A Ratio,0.555556
B/A Size,500x900
Beta,1.7


In [164]:
data_dict

{'Closing Price': ([], '$154.09'),
 "Day's Change": ([], '-2.68| |(-1.71%)'),
 'Bid': ([], '153.57'),
 'Ask': ([], '153.60'),
 'B/A Size': ([], '500x900'),
 "Day's High": ([], '160.02'),
 "Day's Low": ([], '151.20'),
 'Volume': (['(Below Average)',
   'Volume:',
   '49,207,165',
   '10-day average volume:',
   '57,771,720'],
  '49,207,165'),
 'Volume:': ([], '57,771,720'),
 '10-day average volume:': ([], '49,207,165'),
 'Prev Close': ([], '156.77'),
 "Today's Open": ([], '159.66'),
 "Day's Range": ([], '151.20-160.02'),
 'Avg Vol (10-day)': ([], '57.8M'),
 'Last (time)': ([], '4:00p ET 11/18/22'),
 'Last (size)': ([], '5'),
 '52-Wk Range': (['108.13 - 346.47'], 'Low|High'),
 '% Above Low': (['% Above Low',
   'The current share price compared to the 52-week low.'],
  '42.50%'),
 'Historical Volatility': (['Historical Volatility',
   "The volatility of a stock over a given time period. It is calculated by determining the average standard deviation from the average price of the stock ove

In [158]:
alt_info

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['(Below Average)',
  'Volume:',
  '49,207,165',
  '10-day average volume:',
  '57,771,720'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['108.13 - 346.47'],
 ['% Above Low', 'The current share price compared to the 52-week low.'],
 ['Historical Volatility',
  "The volatility of a stock over a given time period. It is calculated by determining the average standard deviation from the average price of the stock over one month or 21 business days. Historical volatility can be compared with implied volatility to determine if a stock's options are over- or undervalued."],
 [],
 [],
 ['EPS (TTM, GAAP)',
  "GAAP EPS (TTM) refers to a company's earnings per share (EPS) for the trailing twelve-month (TTM) period. EPS is calculated by dividing the adjusted income available to common stockholders for the trailing twelve months by the trailing twelve-month diluted weighted average shares outstanding.",
  'GAAP vs. Non-GAAP Earnings',
  'TD Ameritrade displays two 

In [157]:
fields

['Closing Price',
 "Day's Change",
 'Bid',
 'Ask',
 'B/A Size',
 "Day's High",
 "Day's Low",
 'Volume',
 'Volume:',
 '10-day average volume:',
 'Prev Close',
 "Today's Open",
 "Day's Range",
 'Avg Vol (10-day)',
 'Last (time)',
 'Last (size)',
 '52-Wk Range',
 '% Above Low',
 'Historical Volatility',
 'Market Cap',
 'Shares Outstanding',
 'EPS (TTM, GAAP)',
 'P/E Ratio (TTM, GAAP)',
 'Annual Dividend/Yield',
 'Ex-dividend Date',
 'Dividend Pay Date',
 'Beta',
 '% Held by Institutions',
 'Short Interest',
 'Market Edge Opinion:',
 'Score:']

In [155]:
dds

[<dd>$154.09</dd>,
 <dd class="market-hours-change" data-sitevendortrack='{"actionName": "unaffiliatedLinkClick", "details": ["general", "smart text", "days change", "open"]}'><div class="ui-price-change neg"><div class="iconWrapper large floated"><div class="icon icon-arrow-negative"></div></div><span class="price-change">-2.68</span><span class="percent-change"> <span class="neg">(-1.71%)</span></span></div></dd>,
 <dd><a data-sitevendortrack='{"actionName": "actionClick", "details": ["bid"]}' href="https://invest.ameritrade.com/cgi-bin/apps/u/ThirdPartyUrlLauncher/new?target=bidBttn&amp;param=NVDA&amp;param2=153.57" target="vendorLinks">153.57</a></dd>,
 <dd><a data-sitevendortrack='{"actionName": "actionClick", "details": ["ask"]}' href="https://invest.ameritrade.com/cgi-bin/apps/u/ThirdPartyUrlLauncher/new?target=askBttn&amp;param=NVDA&amp;param2=153.6" target="vendorLinks">153.60</a></dd>,
 <dd>500x900</dd>,
 <dd>160.02</dd>,
 <dd>151.20</dd>,
 <dd>49,207,165</dd>,
 <dd>57,771,72

In [151]:
texts

['Closing Price',
 "Day's Change",
 'Bid',
 'Ask',
 'B/A Size',
 "Day's High",
 "Day's Low",
 'Volume|(Below Average)|Volume:|49,207,165|10-day average volume:|57,771,720',
 'Volume:',
 '10-day average volume:',
 'Prev Close',
 "Today's Open",
 "Day's Range",
 'Avg Vol (10-day)',
 'Last (time)',
 'Last (size)',
 '52-Wk Range|108.13 - 346.47',
 '% Above Low|% Above Low|The current share price compared to the 52-week low.',
 "Historical Volatility|Historical Volatility|The volatility of a stock over a given time period. It is calculated by determining the average standard deviation from the average price of the stock over one month or 21 business days. Historical volatility can be compared with implied volatility to determine if a stock's options are over- or undervalued.",
 'Market Cap',
 'Shares Outstanding',
 "EPS (TTM, GAAP)|EPS (TTM, GAAP)|GAAP EPS (TTM) refers to a company's earnings per share (EPS) for the trailing twelve-month (TTM) period. EPS is calculated by dividing the adjus

In [149]:
dts

[<dt>Closing Price</dt>,
 <dt>Day's Change</dt>,
 <dt>Bid</dt>,
 <dt>Ask</dt>,
 <dt>B/A Size</dt>,
 <dt>Day's High</dt>,
 <dt>Day's Low</dt>,
 <dt>Volume<div class="ui-tooltip ui-allow-hover"><a class="volumeDetails ui-allow-hover" data-mouseenteraction="openTooltip" data-mouseleaveaction="openTooltip" href="javascript:void(0);">(Below Average)</a><div class="ui-tooltip-content shadow boxshadow ui-open-down ui-position-right"><div class="volume-details"><dl><dt>Volume:</dt><dd>49,207,165</dd><br/><dt>10-day average volume:</dt><dd>57,771,720</dd></dl></div></div><div class="ui-arrow"></div></div></dt>,
 <dt>Volume:</dt>,
 <dt>10-day average volume:</dt>,
 <dt>Prev Close</dt>,
 <dt>Today's Open</dt>,
 <dt>Day's Range</dt>,
 <dt>Avg Vol (10-day)</dt>,
 <dt>Last (time)</dt>,
 <dt>Last (size)</dt>,
 <dt>52-Wk Range<br/>108.13 - 346.47</dt>,
 <dt><div class="ui-tooltip ui-on-click ui-definition-link"><div class="closer" data-clickaction="openTooltip"></div><a class="definition-link" data-cl

In [81]:
driver.quit()